In [1]:
from qiskit import *
from math import log2, lcm

In [2]:
import math, random, time
from fractions import Fraction
from sympy import primerange, mod_inverse

# ---------- utilitários ----------
def modular_deviation(a, N):
    """Δ_N(L) = |L/N - round(L/N)| usando Fraction para precisão exata"""
    return min(a%N, -a%N)/N

def estimate_required_primes(N, m, ell):
    """Estimativa aproximada de |P| = ceil(m * log2(N) / ell)"""
    nbits = math.log2(N)
    return math.ceil(m * nbits / ell)

# ---------- geração de pool ----------
def generate_primes_up_to_ell_bits(ell, limit_count=None):
    """Gera primos < 2^ell. Limit_count limita quantidade lida (útil para ell grande)."""
    limit = (1 << ell) - 1
    if limit_count is None:
        return list(primerange(2, limit + 1))
    else:
        out = []
        for p in primerange(2, limit + 1):
            out.append(p)
            if len(out) >= limit_count:
                break
        return out

def generate_primes_biased_by_value(ell, top_fraction=0.3, mix_fraction=0.05, limit_count=None):
    """
    Gera pool com viés para os maiores:
    - top_fraction = fração superior do intervalo em valor (ex: 0.25 = últimos 25% valores)
    - mix_fraction = fração dos primos abaixo do threshold a amostrar (para ajuste fino)
    - limit_count = limitar geração total (opcional)
    Retorna pool ordenado decrescentemente (maiores primeiro).
    """
    min_val = 2
    max_val = (1 << ell) - 1
    range_w = max_val - min_val + 1
    threshold_value = max_val - int(range_w * top_fraction) + 1

    all_primes = generate_primes_up_to_ell_bits(ell, limit_count=limit_count)
    big_primes = [p for p in all_primes if p >= threshold_value]
    small_primes = [p for p in all_primes if p < threshold_value]

    mix_count = max(0, int(len(small_primes) * mix_fraction))
    mix_sample = random.sample(small_primes, mix_count) if mix_count > 0 else []

    pool = sorted(big_primes, reverse=True) + sorted(mix_sample, reverse=True)
    # ensure uniqueness
    seen = set()
    pool_unique = []
    for p in pool:
        if p not in seen:
            seen.add(p)
            pool_unique.append(p)
    return pool_unique

# ---------- seleção gulosa ----------
def greedy_select_primes(pool, N, m, overshoot_limit=4):
    """
    Seleciona primos na ordem do pool até atingir L >= N^m.
    Limita overshoot evitando L > N^(m*overshoot_limit).
    Retorna (L, P) ou (None,None).
    """
    target = N ** m
    max_L = N ** (m * overshoot_limit)
    L = 1
    P = []
    for p in pool:
        L *= p
        P.append(p)
        if L >= target:
            if L > max_L and len(P) > 1:
                # remove último e continue
                L //= p
                P.pop()
                continue
            return L, P
    return None, None

# ---------- local search / heurísticas ----------
def local_swap_improve(N, L, P, pool, max_swaps=3, iter_limit=200):
    """
    Tenta melhorar Δ trocando até `max_swaps` primos entre P e candidatos de pool.
    Retorna (best_L, best_P, best_d).
    Usa uma busca estocástica: tenta combinações aleatórias de swaps.
    """
    best_L = L
    best_P = P[:]
    best_d = modular_deviation(L, N)
    candidates = [c for c in pool if c not in P]
    if not candidates:
        return best_L, best_P, best_d

    # tentativa 1: trocar um por um (determinístico)
    for i in range(len(P)-1, max(0, len(P)-7)-1, -1):  # prioriza swaps nos últimos ~7 elementos
        p_old = P[i]
        for cand in candidates:
            L_new = (best_L // p_old) * cand
            d_new = modular_deviation(L_new, N)
            if 0 < d_new < best_d:
                best_d = d_new; best_L = L_new; best_P = P[:]; best_P[i] = cand

    # tentativa estocástica: swaps múltiplos aleatórios
    start_time = time.time()
    tries = 0
    while tries < iter_limit and time.time() - start_time < 2.0:
        tries += 1
        k = random.randint(1, max_swaps)
        idxs = random.sample(range(len(P)), k)
        cand_idxs = random.sample(range(len(candidates)), k if k <= len(candidates) else len(candidates))
        P_new = P[:]
        L_new = best_L
        # perform swaps
        for ii, ci in zip(idxs, cand_idxs):
            old = P_new[ii]
            cand = candidates[ci]
            L_new = (L_new // old) * cand
            P_new[ii] = cand
        d_new = modular_deviation(L_new, N)
        if 0 < d_new < best_d:
            best_d = d_new; best_L = L_new; best_P = P_new[:]
    return best_L, best_P, best_d

# ---------- main search ----------
def find_P_with_robust_search(N, m, f,
                              ell_start=None, ell_max_increase=6,
                              top_fraction=0.3, mix_fraction=0.03,
                              limit_primes_read=None,
                              overshoot_limit=3,
                              greedy_attempts=300,
                              local_iter=300):
    """
    Busca por P tais que 0 < Δ_N(L) <= 2^-f, tentando aumentar ell se necessário.
    Retorna (P,L,delta,ell_used) ou (best_found,...).
    Parâmetros explicados no código acima.
    """
    if ell_start is None:
        ell_start = math.ceil(math.log2(math.log2(N)))
    limiar = Fraction(1, 2**f)
    best_overall = (None, None, Fraction(1,1), None)  # (P,L,d,ell)

    for ell_inc in range(0, ell_max_increase+1):
        ell = ell_start + ell_inc
        print(f"[info] testando ell={ell} ... (estimativa |P| ≈ {estimate_required_primes(N,m,ell)})")
        pool = generate_primes_biased_by_value(ell, top_fraction=top_fraction,
                                               mix_fraction=mix_fraction, limit_count=limit_primes_read)
        if not pool:
            print(f"[warn] nenhum primo gerado para ell={ell}")
            continue

        # check if product of pool can in principle reach target
        prod_all = 1
        for p in pool:
            prod_all *= p
            if prod_all >= N**m:
                break
        if prod_all < N**m:
            print(f"[info] produto de todo pool insuficiente para N^m — aumentando ell")
            continue

        # várias tentativas greedy com pequenas permutações
        for attempt in range(greedy_attempts):
            # partially shuffle lower part to keep bias but explore variations
            head = pool[:max(1, int(len(pool)*0.6))]  # keep top 60% order
            tail = pool[max(1, int(len(pool)*0.6)):]
            random.shuffle(tail)
            candidate_pool = head + tail

            L, P = greedy_select_primes(candidate_pool, N, m, overshoot_limit=overshoot_limit)
            if L is None:
                continue
            d = modular_deviation(L, N)

            # immediate accept if meets strict criterion (and not zero)
            if 0 < d <= limiar:
                print(f"[ok] encontrado ell={ell}, tentativa {attempt+1}, Δ={float(d):.3e}, |P|={len(P)}")
                print(f"Primos encontrados: {P}")
                return P, L, float(d), ell

            # try local improve
            L2, P2, d2 = local_swap_improve(N, L, P, candidate_pool, max_swaps=4, iter_limit=local_iter)
            if 0 < d2 <= limiar:
                print(f"[ok local] encontrado ell={ell}, tentativa {attempt+1}, Δ={float(d2):.3e}, |P|={len(P2)}")
                print(f"Primos encontrados: {P2}")
                return P2, L2, float(d2), ell

            # update best overall if improved
            if d != 0 and d < best_overall[2]:
                best_overall = (P, L, d, ell)

        print(f"[info] não encontrou para ell={ell} em {greedy_attempts} tentativas. melhor Δ até agora = {float(best_overall[2]):.3e}")

    # fim loop ell
    if best_overall[0] is not None:
        Pbest, Lbest, dbest, ellbest = best_overall
        print(f"[best] melhor solução parcialmente encontrada: ell={ellbest}, |P|={len(Pbest)}, Δ={float(dbest):.3e}")
        print(f"Primos encontrados: {Pbest}")
        return Pbest, Lbest, float(dbest), ellbest

    print("[fail] não foi possível encontrar P satisfatório dentro dos limites especificados.")
    return None, None, None, None


if __name__ == "__main__":
    N = 10**12        # N
    m = 8             # número de multiplicações/expoente efetivo
    f = 10            # bits mantidos (limiar 2^-f)
    # Parâmetros de controle da busca:
    ell_start = None          # None -> calcula log2(log2(N))
    ell_max_increase = 6
    top_fraction = 0.25       # priorizar últimos 25% do intervalo [2..2^ell-1]
    mix_fraction = 0.06       # amostra pequena de primos menores para ajuste
    limit_primes_read = None  # limite práticos (ex: 10000) para ell grande
    overshoot_limit = 3
    greedy_attempts = 400
    local_iter = 300

    res = find_P_with_robust_search(N, m, f,
                                    ell_start=ell_start,
                                    ell_max_increase=ell_max_increase,
                                    top_fraction=top_fraction,
                                    mix_fraction=mix_fraction,
                                    limit_primes_read=limit_primes_read,
                                    overshoot_limit=overshoot_limit,
                                    greedy_attempts=greedy_attempts,
                                    local_iter=local_iter)

    P, L, d, ell_used = res
    if P:
        print("\n--- Resultado final ---")
        print("ell_used:", ell_used)
        print("len(P):", len(P))
        print("Δ_N(L):", d)
        print("primos:", P)
        print("L:", L)
    else:
        print("Nenhuma solução encontrada. Tente ajustar parâmetros (ver recomendações).")


[info] testando ell=6 ... (estimativa |P| ≈ 54)
[info] produto de todo pool insuficiente para N^m — aumentando ell
[info] testando ell=7 ... (estimativa |P| ≈ 46)
[info] produto de todo pool insuficiente para N^m — aumentando ell
[info] testando ell=8 ... (estimativa |P| ≈ 40)
[info] produto de todo pool insuficiente para N^m — aumentando ell
[info] testando ell=9 ... (estimativa |P| ≈ 36)
[info] produto de todo pool insuficiente para N^m — aumentando ell
[info] testando ell=10 ... (estimativa |P| ≈ 32)
[ok local] encontrado ell=10, tentativa 2, Δ=2.068e-04, |P|=34
Primos encontrados: [1021, 1019, 1013, 1009, 997, 991, 983, 977, 971, 967, 953, 947, 941, 937, 929, 919, 911, 907, 887, 883, 881, 877, 863, 859, 857, 853, 839, 229, 811, 787, 257, 809, 2, 827]

--- Resultado final ---
ell_used: 10
len(P): 34
Δ_N(L): 0.000206822059
primos: [1021, 1019, 1013, 1009, 997, 991, 983, 977, 971, 967, 953, 947, 941, 937, 929, 919, 911, 907, 887, 883, 881, 877, 863, 859, 857, 853, 839, 229, 811, 787, 

In [3]:
from qiskit import *
from math import log2, lcm, gcd

# ----------- pre-computed values ----------- #

def get_primes(N: int, bits_e: int, f: int):
    P, L, d, max_ell_used = find_P_with_robust_search(N, bits_e, f,
                                    ell_start=None,
                                    ell_max_increase=6,
                                    top_fraction=0.25,
                                    mix_fraction=0.06,
                                    limit_primes_read=None,
                                    overshoot_limit=3,
                                    greedy_attempts=400,
                                    local_iter=300)

    return P, L, d, max_ell_used

def get_constants(L: int, P: list[int], N: int, f: int):
    u = get_ujs(P, L)
    C = get_Cjk(u, N, L, f)
    return u, C


def get_ujs(P, L):
    """
    Calcula os cofatores u_j = (L/p_j) * inv(L/p_j mod p_j) mod L
    """
    if len(set(P)) != len(P):
        raise ValueError("Lista P contém primos repetidos — remova duplicatas antes de calcular u_j.")
    
    u_list = []
    for p in P:
        Lj = L // p
        a = Lj%p
        if a == 0 or gcd(a, p) != 1:
            raise ValueError(f"Não é possível inverter Lj mod p para p={p}. "
                             f"Lj % p == {a}. Verifique se P contém primos distintos e se p é primo.")
        inv = pow(Lj, -1, p)
        u = (Lj * inv)
        u_list.append(u)
    return u_list


def get_Cjk(u_list, N, L, f):
    """
    Calcula os fatores C_{j,k} conforme (eq. 21):
    C_{j,k} = (((u_j << k) mod L mod N) >> t) mod (N >> t)
    """
    C = []
    nbits = int(log2(N)) + 1
    t = nbits - f
    N_trunc = N >> t

    for j, u in enumerate(u_list):
        Cj = []
        for k in range(f):  # bits mais significativos a considerar
            val = ((u << k) % L) % N
            val = val >> t
            val = val % N_trunc
            Cj.append(val)
        C.append(Cj)
    return C


# ----------- quantum alg ----------- #

def naive_bit_shift(shift: int, reg_size: int, left: bool = True):
    """
    shifts the register by (shift) truncating values if necessary,
    shift direction defined by (left)
    """
    quantum_circuit = QuantumCircuit(reg_size)

    if not left:
        for i in range(0, reg_size-shift):
            quantum_circuit.swap(i, shift+i)
        quantum_circuit.reset(range(reg_size-shift,reg_size))
    else:
        for i in range(0, reg_size-shift):
            quantum_circuit.swap(i, shift+i)
        quantum_circuit.reset(range(shift))

    return quantum_circuit

In [4]:
def naive_bit_shift(shift: int, reg_size: int, left: bool = True):
    """
    shifts the register by (shift) truncating values if necessary,
    shift direction defined by (left)
    """
    quantum_circuit = QuantumCircuit(reg_size)

    if not left:
        for i in range(0, reg_size-shift):
            quantum_circuit.swap(i, shift+i)
        quantum_circuit.reset(range(reg_size-shift,reg_size))
    else:
        for i in range(0, reg_size-shift):
            quantum_circuit.swap(i, shift+i)
        quantum_circuit.reset(range(shift))

    return quantum_circuit
naive_bit_shift(2, 4, False).draw("mpl")

In [5]:
"""
Implementação por: Guilherme Fontoura
https://github.com/gdhfontoura
"""



from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library import QFT, PhaseGate, XGate
import numpy as np
from math import pi

def qft_rotations(circuit, n, n_max):
    """Performs qft on the first n qubits in circuit (without swaps)"""
    if n == 0:
        return circuit
    n -= 1

    circuit.h(n)
    for qubit in range(n-1, n):
        if qubit >= 0:
            circuit.cp(pi/2**(n-qubit), qubit, n)
    if n == n_max - 1:
        circuit.p(pi/2**(n_max-2), n)
    # At the end of our function, we call the same function again on
    # the next qubits (we reduced n by one earlier in the function)
    qft_rotations(circuit, n, n_max)
    return circuit

def swap_registers(circuit, n):
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    return circuit

def qft(circuit, n, n_max, do_swaps=False):
    """QFT on the first n qubits in circuit"""
    qft_rotations(circuit, n, n_max)
    if do_swaps:
        swap_registers(circuit, n)
    return circuit

def draper_adder(n_bits, a, controlado=False, div=False, control_number=1):
    """Retorna um circuito que corresponde ao DraperQFTAdder [1], sem as QFTs e com um operando clássicamente calculado.

    Faz a operação a + b : b é o número que está no registrador reg_b.

    Parametros:
    n_bits : int
        Número de bits do operando.
    a : int
        Operando implícito calculado classicamente.
    controlado : bool
        Se o adder será controlado ou não (precisa do bit de controle).
    div : bool
        Se o adder será utilizado em uma divisão.
    control_number : int
        Número de qubits que controlam esse operador (c qubits).

    Retorna:
    QuantumCircuit 
    circuito montado com os registradores nessa ordem:
        if controlado: 
            c + n + 1 qubits
            registrador_controle (c bits)
            registrador_operando (n bits)
            registrador_carryout (1 bit)
        else:
            n + 1 qubits
            registrador_operando (n bits)
            registrador_carryout (1 bit)

    References: 
        [1] T. G. Draper, Addition on a Quantum Computer, 2000. arXiv:quant-ph/0008033
        https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.DraperQFTAdder
    """

    # Registradores principais
    reg_b = QuantumRegister(n_bits, "b")
    reg_cout = QuantumRegister(1, "cout")

    if div: # Se for divisão, *2 até ficar com n_bits
        bitstring = bin(a)[2:] + ("0" * (n_bits - len(bin(a)) + 2))
    else: # Se não for divisão, igualar o numero de bits sem alterar o valor
        bitstring = bin(a)[2:].zfill(n_bits)
    bitstring = bitstring[::-1]

    if not controlado:
        qc = QuantumCircuit(reg_b, reg_cout, name="adapt_drap_adder")
        l = [0]*(n_bits)
        cout = [0]
        for j in range(n_bits):
            for k in range(n_bits - j):
                if bitstring[j] == "1":
                    lam = np.pi / (2**k)
                    l[j + k] += lam

        for j in range(n_bits):
            if bitstring[n_bits - j - 1] == "1":
                lam = np.pi / (2 ** (j + 1))
                cout[0] += lam

        for i in range(len(l)):
            if l[i] != 0:
                qc.p(l[i], reg_b[i])

        for i in range(len(cout)):
            if cout[i] != 0:
                qc.p(cout[0], reg_cout[0])

        # print(l)

    else:
        # Registrador de controle
        reg_c = QuantumRegister(control_number, "c")

        qc = QuantumCircuit(reg_c, reg_b, reg_cout, name="c_adapt_drap_adder")

        l = [0]*(n_bits)
        cout = [0]

        # Portas controladas por A e pelo registrador de controle
        for j in range(n_bits):
            for k in range(n_bits - j):
                if bitstring[j] == "1":
                    lam = np.pi / (2**k)
                    l[j + k] += lam

        for i in range(len(l)):
            if l[i] != 0:            
                qc.append(PhaseGate(l[i]).control(control_number), reg_c[:] + reg_b[i:i + 1])

        for j in range(n_bits):
            if bitstring[n_bits - j - 1] == "1":
                lam = np.pi / (2 ** (j + 1))
                cout[0] += lam
        
        for i in range(len(cout)):
            if cout[i] != 0:
                qc.append(PhaseGate(cout[i]).control(control_number), reg_c[:] + reg_cout[:])

    return qc


def adder_mod(n_bits, a, N, controlado=False, control_number=1):
    """Retorna um circuito que implementa o Adder Modular proposto no artigo [1]
        usando o DraperQFTAdder com 1 operando clássicamente calculado como Adder e aplicando
        otimizações do artigo [2] quanto as QFTs.

    Faz a operação a + b mod N : b é o número que está no registrador reg_b.

    Exemplo: Adder Modular controlado, com um operando de 4 qubits. 

      ┌─────────────────────┐┌────────────────────────┐                                             ┌────────────────────────┐                                ┌─────────────────────┐
   c: ┤0                    ├┤0                       ├─────────────────────────────────────────────┤0                       ├────────────────■───────────────┤0                    ├
      │                     ││                        │┌───────┐     ┌──────┐┌─────────────────────┐│                        │┌───────┐       │       ┌──────┐│                     │
 b_0: ┤1                    ├┤1                       ├┤0      ├─────┤0     ├┤1                    ├┤1                       ├┤0      ├───────┼───────┤0     ├┤1                    ├
      │                     ││                        ││       │     │      ││                     ││                        ││       │       │       │      ││                     │
 b_1: ┤2                    ├┤2                       ├┤1      ├─────┤1     ├┤2                    ├┤2                       ├┤1      ├───────┼───────┤1     ├┤2                    ├
      │  c_adapt_drap_adder ││  c_adapt_drap_adder_dg ││       │     │      ││                     ││  c_adapt_drap_adder_dg ││       │       │       │      ││  c_adapt_drap_adder │
 b_2: ┤3                    ├┤3                       ├┤2 IQFT ├─────┤2 QFT ├┤3                    ├┤3                       ├┤2 IQFT ├───────┼───────┤2 QFT ├┤3                    ├
      │                     ││                        ││       │     │      ││  c_adapt_drap_adder ││                        ││       │       │       │      ││                     │
 b_3: ┤4                    ├┤4                       ├┤3      ├─────┤3     ├┤4                    ├┤4                       ├┤3      ├───────┼───────┤3     ├┤4                    ├
      │                     ││                        ││       │     │      ││                     ││                        ││       │┌───┐  │  ┌───┐│      ││                     │
cout: ┤5                    ├┤5                       ├┤4      ├──■──┤4     ├┤5                    ├┤5                       ├┤4      ├┤ X ├──■──┤ X ├┤4     ├┤5                    ├
      └─────────────────────┘└────────────────────────┘└───────┘┌─┴─┐└──────┘│                     │└────────────────────────┘└───────┘└───┘┌─┴─┐└───┘└──────┘└─────────────────────┘
 anc: ──────────────────────────────────────────────────────────┤ X ├────────┤0                    ├────────────────────────────────────────┤ X ├────────────────────────────────────
                                                                └───┘        └─────────────────────┘                                        └───┘

    Parametros:
    n_bits : int
        Número de bits do operando.
    a : int
        Operando implícito calculado classicamente.
    N : int
        Operando implícito que controla o mod
    controlado : bool
        Se o adder será controlado ou não (precisa do bit de controle).
    control_number : int
        Número de qubits que controlam esse operador (c qubits).

    Retorna:
    QuantumCircuit 
    circuito montado com os registradores nessa ordem:
        if controlado: 
            c + n + 2 qubits
            registrador_controle (c bits)
            registrador_operando (n bits)
            registrador_carryout (1 bit)
            registrador_ancilla (1 bit)
        else:
            n + 2 qubits
            registrador_operando (n bits)
            registrador_carryout (1 bit)
            registrador_ancilla (1 bit)

    References: 
        [1] Vlatko Vedral, Adriano Barenco, and Artur Ekert, Quantum networks for elementary arithmetic operations, quant-ph/9511018
        [2] Stephane Beauregard, Circuit for Shor's algorithm using 2n+3 qubits, arXiv:quant-ph/0205095
    """

    # Construção dos registradores

    if controlado:
        reg_control = QuantumRegister(control_number, "c")

    reg_b = QuantumRegister(n_bits, "b")
        
    reg_cout = QuantumRegister(1, "cout")

    reg_anc = QuantumRegister(1, "anc")

    if controlado:
        if control_number == 1:
            # Construção do circuito
            qc = QuantumCircuit(reg_control, reg_b, reg_cout, reg_anc, name="c_adder_mod") 
        
            qc.append(draper_adder(n_bits, a, controlado=True), reg_control[:] + reg_b[:] + reg_cout[:])

            qc.append(draper_adder(n_bits, N, controlado=True).inverse(), reg_control[:] + reg_b[:] + reg_cout[:])

            qc.append(QFT(n_bits + 1, do_swaps=False).inverse(), reg_b[:] + reg_cout[:])

            qc.cx(reg_cout[0], reg_anc[0])

            qc.append(QFT(n_bits + 1, do_swaps=False), reg_b[:] + reg_cout[:])

            qc.append(draper_adder(n_bits, N, controlado=True), reg_anc[:] + reg_b[:] + reg_cout[:])
        
            qc.append(draper_adder(n_bits, a, controlado=True).inverse(), reg_control[:] + reg_b[:] + reg_cout[:])

            qc.append(QFT(n_bits + 1, do_swaps=False).inverse(), reg_b[:] + reg_cout[:])

            qc.x(reg_cout)
            qc.ccx(reg_control[0], reg_cout[0], reg_anc[0])
            qc.x(reg_cout)

            qc.append(QFT(n_bits + 1, do_swaps=False), reg_b[:] + reg_cout[:])

            qc.append(draper_adder(n_bits, a, controlado=True), reg_control[:] + reg_b[:] + reg_cout[:])
        elif control_number == 2:
            qc = QuantumCircuit(reg_control, reg_b, reg_cout, reg_anc, name="c_adder_mod") 

            qft_simplificada = QuantumCircuit(n_bits + 1)
            qft(qft_simplificada, n_bits + 1, n_bits + 1)
        
            qc.append(draper_adder(n_bits, a, controlado=True, control_number=control_number), reg_control[:] + reg_b[:] + reg_cout[:])

            qc.append(draper_adder(n_bits, N, controlado=True, control_number=control_number).inverse(), reg_control[:] + reg_b[:] + reg_cout[:])

            qc.append(QFT(n_bits + 1, do_swaps=False).inverse(), reg_b[:] + reg_cout[:])
            #qc.append(qft_simplificada.inverse(), reg_b[:] + reg_cout[:])

            qc.cx(reg_cout[0], reg_anc[0])

            qc.append(QFT(n_bits + 1, do_swaps=False), reg_b[:] + reg_cout[:])
            #qc.append(qft_simplificada, reg_b[:] + reg_cout[:])

            qc.append(draper_adder(n_bits, N, controlado=True), reg_anc[:] + reg_b[:] + reg_cout[:])
        
            qc.append(draper_adder(n_bits, a, controlado=True, control_number=control_number).inverse(), reg_control[:] + reg_b[:] + reg_cout[:])

            qc.append(QFT(n_bits + 1, do_swaps=False).inverse(), reg_b[:] + reg_cout[:])
            #qc.append(qft_simplificada.inverse(), reg_b[:] + reg_cout[:])

            qc.x(reg_cout)
            qc.append(XGate().control(control_number+1), reg_control[:] + reg_cout[:] + reg_anc[:])
            qc.x(reg_cout)

            qc.append(QFT(n_bits + 1, do_swaps=False), reg_b[:] + reg_cout[:])
            #qc.append(qft_simplificada, reg_b[:] + reg_cout[:])

            qc.append(draper_adder(n_bits, a, controlado=True, control_number=control_number), reg_control[:] + reg_b[:] + reg_cout[:])


    else:
        qc = QuantumCircuit(reg_b, reg_cout, reg_anc, name="adder_mod")

        qc.append(draper_adder(n_bits, a), reg_b[:] + reg_cout[:])

        qc.append(draper_adder(n_bits, N).inverse(), reg_b[:] + reg_cout[:])

        qc.append(QFT(n_bits + 1, do_swaps=False).inverse(), reg_b[:] + reg_cout[:])

        qc.cx(reg_cout[0], reg_anc[0])

        qc.append(QFT(n_bits + 1, do_swaps=False).inverse(), reg_b[:] + reg_cout[:])

        qc.append(draper_adder(n_bits, N, controlado=True), reg_anc[:] + reg_b[:] + reg_cout[:])
    
        qc.append(draper_adder(n_bits, a).inverse(), reg_b[:] + reg_cout[:])

        qc.append(QFT(n_bits + 1, do_swaps=False).inverse(), reg_b[:] + reg_cout[:])

        qc.cx(reg_cout[0], reg_anc[0], ctrl_state="0")

        qc.append(QFT(n_bits + 1, do_swaps=False).inverse(), reg_b[:] + reg_cout[:])

        qc.append(draper_adder(n_bits, a), reg_b[:] + reg_cout[:])

    return qc

'''
def draper_adder(n_bits, a, controlado=False, kind="half", control_number=1, div=False):
    """Retorna um circuito que corresponde ao DraperQFTAdder [1], sem as QFTs e com um operando clássicamente calculado.

    Faz a operação a + b : b é o número que está no registrador reg_b.

    Parametros:
    n_bits : int
        Número de bits do operando.
    a : int
        Operando implícito calculado classicamente.
    kind : string
        Diz o tipo do Adder, "half" é o default e adiciona um CarryOut, "fixed" não adiciona CarryOut
        e por isso a adição é feita módulo 2^n_bits
    controlado : bool
        Se o adder será controlado ou não (precisa do bit de controle).
    control_number : int
        Número de qubits que controlam esse operador (c qubits).
    div : bool
        Se o adder será utilizado em uma divisão.

    Retorna:
    QuantumCircuit 
    circuito montado com os registradores nessa ordem:
        if controlado: 
            c + n + 1 qubits
            registrador_controle (c bits)
            registrador_operando (n bits)
            registrador_carryout (1 bit)
        else:
            n + 1 qubits
            registrador_operando (n bits)
            registrador_carryout (1 bit)

    References: 
        [1] T. G. Draper, Addition on a Quantum Computer, 2000. arXiv:quant-ph/0008033
        https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.DraperQFTAdder
    """

    # Registradores principais
    reg_b = QuantumRegister(n_bits, "b")
    if kind=="half":
        reg_cout = QuantumRegister(1, "cout")

    if div: # Se for divisão, *2 até ficar com n_bits
        bitstring = bin(a)[2:] + ("0" * (n_bits - len(bin(a)) + 2))
    else: # Se não for divisão, igualar o numero de bits sem alterar o valor
        bitstring = bin(a)[2:].zfill(n_bits)
    bitstring = bitstring[::-1]

    if not controlado:
        if kind=="half":
            qc = QuantumCircuit(reg_b, reg_cout, name="adapt_drap_adder")
        else: 
            qc = QuantumCircuit(reg_b, name="adapt_drap_adder")

        # Portas controladas por A
        for j in range(n_bits):
            for k in range(n_bits - j):
                if bitstring[j] == "1":
                    lam = np.pi / (2**k)
                    qc.p(lam, reg_b[j + k])

        if kind=="half":
            for j in range(n_bits):
                if bitstring[n_bits - j - 1] == "1":
                    lam = np.pi / (2 ** (j + 1))
                    qc.p(lam, reg_cout[0])

    else:
        # Registrador de controle
        reg_c = QuantumRegister(control_number, "c")
        if kind=="half":
            qc = QuantumCircuit(reg_c, reg_b, reg_cout, name="c_adapt_drap_adder")
        else:
            qc = QuantumCircuit(reg_c, reg_b, name="c_adapt_drap_adder")

        # Portas controladas por A e pelo registrador de controle
        for j in range(n_bits):
            for k in range(n_bits - j):
                if bitstring[j] == "1":
                    lam = np.pi / (2**k)
                    qc.append(PhaseGate(lam).control(control_number), reg_c[:] + reg_b[j + k:j + k + 1])

        if kind=="half":
            for j in range(n_bits):
                if bitstring[n_bits - j - 1] == "1":
                    lam = np.pi / (2 ** (j + 1))
                    qc.append(PhaseGate(lam).control(control_number), reg_c[:] + reg_cout[:])

    return qc


def adder_mod(n_bits, a, N, controlado=False, control_number=1):
    """Retorna um circuito que implementa o Adder Modular proposto no artigo [1]
        usando o DraperQFTAdder com 1 operando clássicamente calculado como Adder e aplicando
        otimizações do artigo [2] quanto as QFTs.

    Faz a operação a + b mod N : b é o número que está no registrador reg_b.

    Exemplo: Adder Modular controlado, com um operando de 4 qubits. 

      ┌─────────────────────┐┌────────────────────────┐                                             ┌────────────────────────┐                                ┌─────────────────────┐
   c: ┤0                    ├┤0                       ├─────────────────────────────────────────────┤0                       ├────────────────■───────────────┤0                    ├
      │                     ││                        │┌───────┐     ┌──────┐┌─────────────────────┐│                        │┌───────┐       │       ┌──────┐│                     │
 b_0: ┤1                    ├┤1                       ├┤0      ├─────┤0     ├┤1                    ├┤1                       ├┤0      ├───────┼───────┤0     ├┤1                    ├
      │                     ││                        ││       │     │      ││                     ││                        ││       │       │       │      ││                     │
 b_1: ┤2                    ├┤2                       ├┤1      ├─────┤1     ├┤2                    ├┤2                       ├┤1      ├───────┼───────┤1     ├┤2                    ├
      │  c_adapt_drap_adder ││  c_adapt_drap_adder_dg ││       │     │      ││                     ││  c_adapt_drap_adder_dg ││       │       │       │      ││  c_adapt_drap_adder │
 b_2: ┤3                    ├┤3                       ├┤2 IQFT ├─────┤2 QFT ├┤3                    ├┤3                       ├┤2 IQFT ├───────┼───────┤2 QFT ├┤3                    ├
      │                     ││                        ││       │     │      ││  c_adapt_drap_adder ││                        ││       │       │       │      ││                     │
 b_3: ┤4                    ├┤4                       ├┤3      ├─────┤3     ├┤4                    ├┤4                       ├┤3      ├───────┼───────┤3     ├┤4                    ├
      │                     ││                        ││       │     │      ││                     ││                        ││       │┌───┐  │  ┌───┐│      ││                     │
cout: ┤5                    ├┤5                       ├┤4      ├──■──┤4     ├┤5                    ├┤5                       ├┤4      ├┤ X ├──■──┤ X ├┤4     ├┤5                    ├
      └─────────────────────┘└────────────────────────┘└───────┘┌─┴─┐└──────┘│                     │└────────────────────────┘└───────┘└───┘┌─┴─┐└───┘└──────┘└─────────────────────┘
 anc: ──────────────────────────────────────────────────────────┤ X ├────────┤0                    ├────────────────────────────────────────┤ X ├────────────────────────────────────
                                                                └───┘        └─────────────────────┘                                        └───┘

    Parametros:
    n_bits : int
        Número de bits do operando.
    a : int
        Operando implícito calculado classicamente.
    N : int
        Operando implícito que controla o mod
    controlado : bool
        Se o adder será controlado ou não (precisa do bit de controle).
    control_number : int
        Número de qubits que controlam esse operador (c qubits).

    Retorna:
    QuantumCircuit 
    circuito montado com os registradores nessa ordem:
        if controlado: 
            c + n + 2 qubits
            registrador_controle (c bits)
            registrador_operando (n bits)
            registrador_carryout (1 bit)
            registrador_ancilla (1 bit)
        else:
            n + 2 qubits
            registrador_operando (n bits)
            registrador_carryout (1 bit)
            registrador_ancilla (1 bit)

    References: 
        [1] Vlatko Vedral, Adriano Barenco, and Artur Ekert, Quantum networks for elementary arithmetic operations, quant-ph/9511018
        [2] Stephane Beauregard, Circuit for Shor's algorithm using 2n+3 qubits, arXiv:quant-ph/0205095
        [3] Takahashi, Yasuhiro and Kunihiro, Noboru, A quantum circuit for shor's factoring algorithm using 2n + 2 qubits, https://dl.acm.org/doi/abs/10.5555/2011665.2011669
    """

    # Construção dos registradores

    if controlado:
        reg_control = QuantumRegister(control_number, "c")

    reg_b = QuantumRegister(n_bits, "b")

    reg_anc = QuantumRegister(1, "anc")

    if controlado:
            qc = QuantumCircuit(reg_control, reg_b, reg_anc, name="c_adder_mod") 

            qc.append(draper_adder(n_bits, N - a, controlado=True, kind="fixed", control_number=control_number).inverse(), reg_control[:] + reg_b[:])

            qc.append(QFT(n_bits, do_swaps=False).inverse(), reg_b[:])

            qc.cx(reg_b[-1], reg_anc[0])

            qc.append(QFT(n_bits, do_swaps=False), reg_b[:])

            qc.append(draper_adder(n_bits, N, controlado=True, kind="fixed"), reg_anc[:] + reg_b[:])
        
            qc.append(draper_adder(n_bits, a, controlado=True, kind="fixed", control_number=control_number).inverse(), reg_control[:] + reg_b[:])

            qc.append(QFT(n_bits, do_swaps=False).inverse(), reg_b[:])

            qc.x(reg_b[-1])
            qc.append(XGate().control(control_number+1), reg_control[:] + reg_b[-1:] + reg_anc[:])
            qc.x(reg_b[-1])

            qc.append(QFT(n_bits, do_swaps=False), reg_b[:])

            qc.append(draper_adder(n_bits, a, controlado=True, kind="fixed", control_number=control_number), reg_control[:] + reg_b[:])
        
    else:
        qc = QuantumCircuit(reg_b, reg_anc, name="adder_mod")

        qc.append(draper_adder(n_bits, N - a, kind="fixed").inverse(), reg_b[:])

        qc.append(QFT(n_bits, do_swaps=False).inverse(), reg_b[:])

        qc.cx(reg_b[-1], reg_anc[0])

        qc.append(QFT(n_bits, do_swaps=False).inverse(), reg_b[:])

        qc.append(draper_adder(n_bits, N, controlado=True, kind="fixed"), reg_anc[:] + reg_b[:])
    
        qc.append(draper_adder(n_bits, a, kind="fixed").inverse(), reg_b[:])

        qc.append(QFT(n_bits, do_swaps=False).inverse(), reg_b[:])

        qc.cx(reg_b[-1], reg_anc[0], ctrl_state="0")

        qc.append(QFT(n_bits, do_swaps=False).inverse(), reg_b[:])

        qc.append(draper_adder(n_bits, a, kind="fixed"), reg_b[:])

    return qc
'''

'\ndef draper_adder(n_bits, a, controlado=False, kind="half", control_number=1, div=False):\n    """Retorna um circuito que corresponde ao DraperQFTAdder [1], sem as QFTs e com um operando clássicamente calculado.\n\n    Faz a operação a + b : b é o número que está no registrador reg_b.\n\n    Parametros:\n    n_bits : int\n        Número de bits do operando.\n    a : int\n        Operando implícito calculado classicamente.\n    kind : string\n        Diz o tipo do Adder, "half" é o default e adiciona um CarryOut, "fixed" não adiciona CarryOut\n        e por isso a adição é feita módulo 2^n_bits\n    controlado : bool\n        Se o adder será controlado ou não (precisa do bit de controle).\n    control_number : int\n        Número de qubits que controlam esse operador (c qubits).\n    div : bool\n        Se o adder será utilizado em uma divisão.\n\n    Retorna:\n    QuantumCircuit \n    circuito montado com os registradores nessa ordem:\n        if controlado: \n            c + n + 1 q

In [6]:
from qiskit import *

import qiskit

#returns a list with the indexes of all qubits in qregs
def get_qubits(qc: qiskit.QuantumCircuit, qregs: list[qiskit.QuantumRegister]) -> list[int]:
    """Find the index location in the circuit of all qubits from all given registers.
    
    Args:
        qc (QuantumCircuit): The quantum circuit from wich the registers will be searched.
        qregs (list[QuantumRegister]): A list of quantum registers from qc, order does matter.

    Returns:
        indexList(list[int]): A list containing the interger indexes of all qubits from given registers.
    
    Examples:
        Getting the qubits from 2 registers::

            from qiskit import *
            
            a = QuantumRegister(2, "a")

            b = QuantumRegister(2, "b")

            c = QuantumRegister(2, "c")

            qc = QuantumCircuit(a, b, c)

            #Get qubits from c and a
            
            bitIndexes = get_qubits(qc, [c, a])

            #bitIndexes will look like this: [4,5,0,1]
    
    """
    bits = []
    for qreg in qregs:
        n = qc.find_bit(qreg[0]).index
        bits.extend(list(range(n, n+qreg.size)))
    return bits

#applies a not in all bits of a given register acconding to control bit
def cx_reg(num_qubits: int) -> qiskit.QuantumCircuit:
    reg = qiskit.QuantumRegister(num_qubits, name="reg")
    c = qiskit.QuantumRegister(1, name="c")
    qc = qiskit.QuantumCircuit(reg,c)
    qc.name = "cx-reg"
    for qubit in reg:
        qc.cx(c[0], qubit)
    return qc

#depending on the control bit state, copies n qubit register into another
def c_copy(num_qubits: int) -> qiskit.QuantumCircuit:
    """Depending on control-qubit state, copies n-qubit register into another.

    The circuit has the following effect when control-qubit is positive:

    |a,0> -> |a,a>
    
    Args:
        num_qubits (int): the number of qubits in register.

    Returns:
        quantum_curcuit(QuantumCircuit): A quantum circuit defining the operation.
    
    Exemple:
        c_copy(2)

        c:  --●--●---
        
        o0: --●--●---
              
        o1: --|--|---
              
        t0: --⨁-|---
               
        t1: -----⨁---
    """
    c = qiskit.QuantumRegister(1, name="c")
    origin = qiskit.QuantumRegister(num_qubits, name="o")
    target = qiskit.QuantumRegister(num_qubits, name="t")
    quantum_circuit = qiskit.QuantumCircuit(c, origin, target)
    quantum_circuit.name = "c-copy"
    for i in range(num_qubits):
        quantum_circuit.ccx(c[0], origin[i], target[i])
    return quantum_circuit

#if control qubit is 0, zeroes out N, N is given and computed classically, else does nothing
#if N is initialized at 0, sets 0 to N
def c_set_reset(num_qubits: int, N: int) -> qiskit.QuantumCircuit:
    """Depending on control-qubit state, resets register to `0` or does nothing
    
    The circuit has the following effect: 

    c = 0
    -
    |num> -> |0>
    
    |0> -> |num>
    
    c = 1
    -
    |num> -> |num>

    .. note::
        In particular, this circuits control-bit is controlled _negatively_.
        Keep that in mind if you want to use it and make sure to do the proper 
        not opperations to have it positively controlled.
        
        Moreover, :attr:`N` should have no more than :attr:`num_qubits` in its binary representation.

    Args:
        num_qubits (int): the number of qubits in register.
        N (int): the number in wich the register should be set/reseted to.

    Returns:
        quantum_curcuit(QuantumCircuit): A quantum circuit defining the operation.
    """   
    bit_string = bin(N)[2:]
    bit_string = bit_string.rjust(num_qubits, "0")
    circ_n = qiskit.QuantumRegister(num_qubits, name="N")
    c = qiskit.QuantumRegister(1, name="c")
    quantum_circuit = qiskit.QuantumCircuit(c, circ_n)
    quantum_circuit.name = "c-set-reset"
    quantum_circuit.x(c[0])
    for bit in range(len(bit_string)):
        if bit_string[bit] == "1":
            quantum_circuit.cx(c[0], circ_n[num_qubits-bit-1])
    quantum_circuit.x(c[0])
            
    return quantum_circuit


def set_reset_to(num_qubits: int, number: int) -> qiskit.QuantumCircuit:
    """Sets register to :attr:`number` or resets it to `0` if register contains :attr:`number`

    The circuit has the following effect: 

    |0> -> |number>

    |number> -> |0>
    
    .. note::
        :attr:`number` should have no more than :attr:`num_qubits` in its binary representation.
    
    Args:
        num_qubits (int): the number of qubits in register.
        number (int): the number in wich the register should be set/reseted to.

    Returns:
        quantum_curcuit(QuantumCircuit): A quantum circuit defining the operation.
    """
    num = qiskit.QuantumRegister(num_qubits, name="num")
    quantum_curcuit = qiskit.QuantumCircuit(num)
    quantum_curcuit.name = "set-reset"
    bit_string = bin(number)[2:]
    bit_string = bit_string.rjust(num_qubits, "0")
    for bit in range(len(bit_string)):
        if bit_string[bit] == "1":
            quantum_curcuit.x(num[num_qubits-bit-1])
    return quantum_curcuit


def cc_set_reset_to_num(num_qubits: int, num: int) -> qiskit.QuantumCircuit:
    """Depending on control-qubits state, sets register to :attr:`num` or resets it to `0` if register contains :attr:`num`

    The circuit has the following effect on register when both control-qubits are positive: 

    |0> -> |num>

    |num> -> |0>

    .. note::
        :attr:`num` should have no more than :attr:`num_qubits` in its binary representation.
      
    Args:
        num_qubtis (int): the number of qubits in register.
        num (int): the number in wich the register should be set/reseted to.

    Returns:
        quantum_curcuit(QuantumCircuit): A quantum circuit defining the operation.
    """
    c = qiskit.QuantumRegister(1, name="c")
    xi = qiskit.QuantumRegister(1, name="xi")
    target = qiskit.QuantumRegister(num_qubits, name="t")
    quantum_circuit = qiskit.QuantumCircuit(c, xi, target)
    quantum_circuit.name = "cc-set-reset"
    bit_string = bin(num)[2:]
    bit_string = bit_string.rjust(num_qubits, "0")
    for bit in range(len(bit_string)):
        if bit_string[bit] == "1":
            quantum_circuit.ccx(c[0], xi[0], target[num_qubits-bit-1])
    return quantum_circuit

#swaps two registers
def swapper(num_qubits: int) -> qiskit.QuantumCircuit:
    """Swaps bits from two n-qubit registers.

    Defines the operation |x,y> -> |y,x>

    Args:
        num_qubits (int): number of qubits from registers

    Returns:
        quantum_circuit(QuantumCircuit): A quantum circuit defining the swap operation
    
    Example:
        swapper(2)

        x0: --x---- y0

        x1: --|-x-- y1

        y0: --x-|-- x0

        y1: ----x-- x1
    """
    x = qiskit.QuantumRegister(num_qubits, name="x")
    y = qiskit.QuantumRegister(num_qubits, name="y")
    quantum_citcuit = qiskit.QuantumCircuit(x,y)
    quantum_citcuit.name = "swapper"

    for i in range(num_qubits):
        quantum_citcuit.swap(x[i], y[i])
    return quantum_citcuit

def mod_inverse(base, exponent, mod):
    """Compute the modular inverse of base^(2^exponent) mod mod"""
    
    # Compute a^(2^i) using modular exponentiation
    value = pow(base, 2**exponent, mod)

    # Compute modular inverse using Extended Euclidean Algorithm
    def extended_gcd(a, b):
        if b == 0:
            return a, 1, 0
        g, x1, y1 = extended_gcd(b, a % b)
        return g, y1, x1 - (a // b) * y1
    
    g, inverse, _ = extended_gcd(value, mod)

    if g != 1:
        raise ValueError(f"No modular inverse exists for {value} mod {mod}")
    
    return inverse % mod


def qc_MAJ() -> QuantumCircuit:
    """
    
    q0 -----⨁--●-----
            |   |
    q1 --⨁-|---●-----
         |  |   |
    q2 --●--●---⨁----

    Returns:
        quantum_circuit (QuantumCircuit): the circuit implementing the operation.
    """
    quantum_circuit = QuantumCircuit(3)
    quantum_circuit.name = "MAJ"
    quantum_circuit.cx(2, 1)
    quantum_circuit.cx(2, 0)
    quantum_circuit.ccx(0, 1, 2)
    return quantum_circuit

#if argument is true, implements the 2cnot version, else, implements the 3cnot version
#check reference
def qc_UMA(two_version: bool=True) -> QuantumCircuit:
    """
    
    - 2-CNOT verison:

    q0 --●--⨁--●--
         |  |  |
    q1 --●--|--⨁--
         |  |
    q2 --⨁--●-----


    - 3-CNOT version:

    q0 ------●--●------⨁-----
             |  |      |
    q1 --⨁--⨁--●--⨁--|--⨁--
                |      |  |
    q2 ---------⨁-----●--●---


    Returns:
        quantum_circuit (QuantumCircuit): the circuit implementing the operation.
    """
    quantum_circuit = QuantumCircuit(3)
    quantum_circuit.name = "UMA"
    if two_version:
        quantum_circuit.ccx(0, 1, 2)
        quantum_circuit.cx(2, 0)
        quantum_circuit.cx(0, 1)
    else:
        quantum_circuit.x(1)
        quantum_circuit.cx(0, 1)
        quantum_circuit.ccx(0, 1, 2)
        quantum_circuit.x(1)
        quantum_circuit.cx(2, 0)
        quantum_circuit.cx(2, 1)
    return quantum_circuit


def adder_CDKM(num_qubits: int, modulo_2n: bool=False) -> QuantumCircuit:
    """

    Complexity:
    -
    For space, assuming `n` as the number of bits to encode the largest operand, we will have a total of:

    - `1` bit for carryIn/Ancilla
    - `2n` bits for operands `a` and `b`
    - If operation is not `modulo 2^n`:
        - `1` bit for carryOut

    Args:
        num_qubits (int): number of bits from operands.
        modulo_2n (bool): indicates if operation is made modulo 2^n or not.

    Returns:
        quantum_circuit (QuantumCircuit): the circuit implementing the operation.

    Reference:
    -
        A new quantum ripple-carry addition circuit. 
        Steven A. Cuccaro, Thomas G. Draper, Samuel A. Kutin, David Petrie Moulton
    """
    c = QuantumRegister(1, name="c")
    a = QuantumRegister(num_qubits, name="a")
    b = QuantumRegister(num_qubits, name="b")
    if not modulo_2n:
        z = QuantumRegister(1, name="z") 
        quantum_circuit = QuantumCircuit(c,a,b,z, name="Adder-CDKM")
    else:
        quantum_circuit = QuantumCircuit(c,a,b, name="Adder-CDKM-MOD2^n")
    
    quantum_circuit.append(qc_MAJ(), c[0:1] + b[0:1] + a[0:1])

    for i in range(1, num_qubits):
        quantum_circuit.append(qc_MAJ(), a[i-1:i] + b[i:i+1] + a[i:i+1])

    if not modulo_2n: quantum_circuit.cx(a[-1], z[0])
        
    for i in range(num_qubits-1, 0, -1):
        quantum_circuit.append(qc_UMA(), a[i-1:i] + b[i:i+1] + a[i:i+1])
        
    quantum_circuit.append(qc_UMA(), c[0:1] + b[0:1] + a[0:1])
    return quantum_circuit


def mod_adder_CDKM_VBE(num_qubits: int, N: int) -> QuantumCircuit:
    anc = QuantumRegister(1, name="anc")
    a = QuantumRegister(num_qubits, name="a")
    b = QuantumRegister(num_qubits, name="b")
    cO = QuantumRegister(1, name="cO")
    n = QuantumRegister(num_qubits, name="N")
    help = QuantumRegister(1, name="help")
    quantum_circuit = QuantumCircuit(anc, a, b, cO, n, help, name=f"AdderMod{N}-CDKM-VBE")

    quantum_circuit.append(set_reset_to(num_qubits, N), n[:])

    quantum_circuit.append(adder_CDKM(num_qubits), anc[:] + a[:] + b[:] + cO[:])
    quantum_circuit.append(adder_CDKM(num_qubits).inverse(), anc[:] + n[:] + b[:] + cO[:])

    quantum_circuit.cx(cO[0], help[0])

    quantum_circuit.append(c_set_reset(num_qubits, N), help[:] + n[:])
    quantum_circuit.append(adder_CDKM(num_qubits), anc[:] + n[:] + b[:] + cO[:])
    quantum_circuit.append(c_set_reset(num_qubits, N), help[:] + n[:])

    quantum_circuit.append(adder_CDKM(num_qubits).inverse(), anc[:] + a[:] + b[:] + cO[:])

    quantum_circuit.x(cO[0])
    quantum_circuit.cx(cO[0], help[0])
    quantum_circuit.x(cO[0])

    quantum_circuit.append(adder_CDKM(num_qubits), anc[:] + a[:] + b[:] + cO[:])

    quantum_circuit.append(set_reset_to(num_qubits, N), n[:])

    return quantum_circuit

In [7]:
from typing import List
import re
import copy
import time
from pathlib import Path
import numpy as np
from itertools import product
from math import log2
from enum import Enum

"""
Arquivo para realizacao da sintese do circuito feito pelo Raphael Bernadino
Implementacao original: https://github.com/raphaelbernardino/rblk
"""


class Alvo:
    def __init__(self, linha, mapa=None):
        self.linha_original = int(linha)

        if mapa is None:
            self.linha = int(linha)
        else:
            if linha >= len(mapa):
                self.linha = linha
            else:
                self.linha = int(mapa[linha])

        if self.linha < 0:
            raise Exception('Alvo inválido! Linha negativa.')

    def modifica_linha(self, modificacao: int):
        """
        Modifica a linha atual. Para aumentar a linha informe um valor positivo, e para diminuir, um valor negativo.

        modificacao: int
        """
        self.linha = self.linha + modificacao

    def remapeia(self, mapa):
        self.linha = mapa[self.linha]

    def __repr__(self):
        return f'b{self.linha}'

    def __eq__(self, other):
        if not isinstance(other, self.__class__):
            return False

        if self.linha != other.linha:
            return False

        return True

    def __lt__(self, other):
        if not isinstance(other, self.__class__):
            raise Exception('Não é possível comparar Alvo com outro tipo.')

        return self.linha < other.linha

    def __hash__(self):
        return id(self.linha) * hash(self.linha + 1)



class States(Enum):
    alvo = 'av'
    ctrl_ausente = None
    ctrl_positivo = True
    ctrl_negativo = False

    def __eq__(self, other):
        other_value = other

        if type(other_value) is self:
            other_value = other_value.value

        return other_value == self.value

    def __repr__(self):
        if self.value == self.ctrl_positivo:
            return 'Positivo'

        if self.value == self.ctrl_negativo:
            return 'Negativo'

        if self.value == self.ctrl_ausente:
            return 'Ausente'

        # return str(self.value)
        return 'Alvo'

    def __str__(self):
        return self.__repr__()
    

class Controle:
    def __init__(self, linha, sinal, mapa=None):
        self.linha_original = int(linha)

        if mapa is None:
            self.linha = int(linha)
        else:
            if linha >= len(mapa):
                self.linha = linha
            else:
                self.linha = int(mapa[linha])

        if isinstance(sinal, bool):
            self.sinal = sinal

        elif isinstance(sinal, States):
            self.sinal = bool(sinal.value)

        else:
            raise Exception('Sinal inválido! Não é booleano.')

        if self.linha < 0:
            raise Exception('Controle inválido! Linha negativa.')

    def modifica_linha(self, modificacao: int):
        """
        Modifica a linha atual. Para aumentar a linha informe um valor positivo, e para diminuir, um valor negativo.

        modificacao: int
        """
        self.linha = self.linha + modificacao

    def remapeia(self, mapa):
        self.linha = mapa[self.linha]

    def eh_sinal_positivo(self):
        return self.sinal == States.ctrl_positivo

    def eh_sinal_negativo(self):
        return self.sinal == States.ctrl_negativo

    def inverte_sinal(self):
        self.sinal = not self.sinal

    def possui_sinal_inverso(self, other):
        if not isinstance(other, self.__class__):
            return False

        if self.linha != other.linha:
            return False

        if self.sinal == other.sinal:
            return False

        return True

    def __repr__(self):
        s = "" if self.sinal else "'"
        return f'b{self.linha}{s}'

    def __eq__(self, other):
        if not isinstance(other, self.__class__):
            return False

        if self.sinal != other.sinal:
            return False

        if self.linha != other.linha:
            return False

        return True

    def __lt__(self, other):
        if not isinstance(other, self.__class__):
            raise Exception('Não é possível comparar Controle com outro tipo.')

        return self.linha < other.linha

    def __hash__(self):
        return id(self.sinal) * hash(self.linha + 1)


class Toffoli:
    def __init__(self, alvos: List[Alvo], controles: List[Controle], mapa=None):
        self.tamanho_bits = 1 + len(controles)

        self.mapeamento = mapa
        if mapa is None:
            # k = [(i, i) for i in range(self.tamanho_bits)]

            # busca a maior linha de alvo
            maior_linha_alvo = 0
            if isinstance(alvos, list):
                maior_linha_alvo = alvos[0].linha
            elif isinstance(alvos, Alvo):
                maior_linha_alvo = alvos.linha
            else:
                raise Exception(f'Alvo não possui um tipo identificavel. (alvos: {type(alvos)})')

            # busca a maior linha de controle
            maior_linha_controle = 0
            if len(controles) > 0:
                maior_linha_controle = max([ctrl.linha for ctrl in controles])

            # identifica a maior linha entre controles e alvos
            maior_linha = max(maior_linha_alvo, maior_linha_controle)
            # print(f'Maior linha:: {maior_linha_alvo} :: {maior_linha_controle} ==> {maior_linha}')

            # gera o mapeamento a partir da maior linha
            k = [(i, i) for i in range(maior_linha + 1)]
            self.mapeamento = dict(k)

        self.alvos = self.__inicializa_alvos(alvos)
        self.controles = self.__inicializa_controles(controles)

    # def mapeia(self, mapa):
    #     # realiza mapeamento do alvo
    #     self.alvos.mapeia(mapa)
    #
    #     # realiza mapeamento dos controles
    #     for controle in self.controles:
    #         controle.mapeia(mapa)

    def modifica_linhas(self, mapeamento: dict):
        """
        Modifica a linha atual através de mapeamento.

        @param mapeamento: Um dicionário contendo o mapeamento a ser realizado.
        """
        linha_alvo = self.alvos.linha
        nova_linha_alvo = mapeamento[linha_alvo]

        diferenca_linha = linha_alvo - nova_linha_alvo
        self.alvos.modifica_linha(diferenca_linha)

        for ctrl in self.controles:
            linha_ctrl = ctrl.linha
            nova_linha_ctrl = mapeamento[linha_ctrl]

            diferenca_linha = linha_ctrl - nova_linha_ctrl
            ctrl.modifica_linha(diferenca_linha)

    def __inicializa_alvos(self, alvos):
        alvo = list()

        if isinstance(alvos, Alvo):
            um_alvo = copy.deepcopy(alvos)
            um_alvo.linha_original = alvos.linha
            um_alvo.linha = self.mapeamento[alvos.linha]
            alvo.append(um_alvo)

        if isinstance(alvos, list):
            # alvo = [copy.deepcopy(a) for a in alvos]
            for a in alvos:
                um_alvo = copy.deepcopy(a)
                um_alvo.linha_original = a.linha
                # print(f'Mapeamento: {self.mapeamento}')
                # print(f'Alvos: {alvos}')
                # print(f'Linha: {a.linha}')
                # print(f'um_alvo: {um_alvo} :: {um_alvo.linha} :: {um_alvo.linha_original}')
                um_alvo.linha = self.mapeamento[a.linha]
                alvo.append(um_alvo)

        if isinstance(alvos, int):
            um_alvo = Alvo(alvos, self.mapeamento)
            alvo.append(um_alvo)

        if len(alvo) != 1:
            raise Exception('Alvos inválidos! É necessário apenas um alvo.')

        return alvo[0]

    def __inicializa_controles(self, ctrls: List[Controle]):
        controles = list()

        for ctrl in ctrls:
            um_controle = copy.deepcopy(ctrl)
            # print(f'Mapeamento: {self.mapeamento}')
            # print(f'Linha Original: {ctrl.linha} :: {um_controle.linha_original} :: {um_controle.linha}')
            um_controle.linha_original = ctrl.linha
            um_controle.linha = self.mapeamento[ctrl.linha]
            controles.append(um_controle)

        return controles

    def remapeia(self, mapa):
        self.alvos.remapeia(mapa)

        for ctrl in self.controles:
            ctrl.remapeia(mapa)

    def adiciona_controles(self, controles):
        # linhas_usadas = self.obtem_linhas_de_controle_usadas()
        linhas_usadas = self.obtem_linhas_usadas()
        controles_antigos = copy.deepcopy(self.controles)

        for ctrl in controles:
            if ctrl.linha in linhas_usadas:
                self.controles = controles_antigos
                raise Exception('Linha {ctrl.linha} já está ocupada')

            um_controle = copy.deepcopy(ctrl)
            self.controles.append(um_controle)
            self.tamanho_bits = len(self.obtem_linhas_usadas())

    def remove_controle(self, ctrl):
        if ctrl in self.controles:
            self.controles.remove(ctrl)
            self.tamanho_bits = len(self.obtem_linhas_usadas())

    def remove_controles(self, controles):
        for ctrl in controles:
            self.remove_controle(ctrl)

    def obtem_alvos(self):
        return [self.alvos]

    def obtem_copia(self):
        return copy.deepcopy(self)

    def calcula_custo_cnot(self):
        """
        Calcula o custo de CNOTs a partir da quantidade de controles usados.
        Usando como referência o artigo: https://iopscience.iop.org/article/10.1088/2058-9565/acaf9d/meta
        Automatic generation of Grover quantum oracles for arbitrary data structures. Raphael Seidel et al. (2023).

        @return: O custo de CNOTs da porta.
        """
        n = len(self.controles)
        return 2 * n ** 2 - 2 * n + 1

    def calcula_custo_quantico(self):
        """
            https://reversiblebenchmarks.github.io/definitions.html
            https://physics.stackexchange.com/a/236054
            Rahman, Md Zamilur, and Jacqueline E. Rice. "Templates for positive and negative control Toffoli networks."
            International Conference on Reversible Computation. Springer, Cham, 2014.
                https://opus.uleth.ca/bitstream/handle/10133/3727/RAHMAN_MD_ZAMILUR_MSC_2015.pdf
            
            -- POSITIVE --
            n = 1 ==> QC =   1
            n = 2 ==> QC =   1
            n = 3 ==> QC =   5
            n = 4 ==> QC =  13
            n = 5 ==> QC =  29
            n = 6 ==> QC =  61
            n = 7 ==> QC = 125
            n = 8 ==> QC = 253
            n = 9 ==> QC = 509
            n > 9 ==> QC = 2**n - 3
            
            -- NEGATIVE -- 
            n = 1 ==> QC =   1
            n = 2 ==> QC =   1+2
            n = 3 ==> QC =   5+1    (see below)
            n = 4 ==> QC =  13+2
            n = 5 ==> QC =  29+2
            n = 6 ==> QC =  61+2
            n = 7 ==> QC = 125+2
            n = 8 ==> QC = 253+2
            n = 9 ==> QC = 509+2
            n > 9 ==> QC = 2**n - 1
            
            -- observation:: n = 3
            == Rahman, Md Zamilur, and Jacqueline E. Rice (2.8.3. Quantum Cost)
            The cost of T (a, b, c) T (a, b) (Peres gate) or T (a, b) T (a, b, c) 
            (inverse of Peres gate) is set to be 4 instead of 5 + 1 = 6, as the 
            quantum implementation of each of these patterns was found with cost 4.
        """
        custo_total = 0
        n = self.tamanho_bits

        if n == 1:
            # not
            custo_total += 1

        elif n == 3:
            # ccnot
            custo_total += 5

            if self.todos_controles_negativos():
                custo_total += 1

        else:
            # toffoli generalizada
            custo_total = 2 ** n - 3

            if self.todos_controles_negativos():
                custo_total += 2

        return custo_total

    def todos_controles_negativos(self):
        for ctrl in self.controles:
            # se o sinal é positivo, retorna False
            if ctrl.eh_sinal_positivo():
                return False

        # se todos os sinais são negativos, retorna True
        return True

    def eh_porta_not(self):
        return len(self.controles) == 0

    def eh_adjacente(self, other):
        if self.obtem_alvos() != other.obtem_alvos():
            return False

        diff = self.diferenca_entre_controles(other)
        diffs = len(diff)

        # pode diferir apenas em uma linha; se não houver ctrls diff, não é
        if diffs == 0:
            return False

        # se  houver apenas um controle diff, é adjacente
        if diffs == 1:
            return True

        # se houver exatamente dois controles, verifique as linhas
        if diffs == 2:
            gc = diff[0]
            hc = diff[1]

            return gc.linha == hc.linha

        # se houver mais de 2 ctrls diff. não pode ser adjacente
        return False

    def diferenca_entre_controles(self, other):
        s1 = set(self.controles)
        s2 = set(other.controles)
        diff = s1.symmetric_difference(s2)
        return sorted(diff)

    def obtem_linhas_de_alvo_usadas(self):
        linhas_usadas = set()
        linhas_usadas.add(self.alvos.linha)
        return linhas_usadas

    def obtem_linhas_de_controle_usadas(self):
        linhas_usadas = set()

        for ctrl in self.controles:
            linha = ctrl.linha
            linhas_usadas.add(linha)

        return linhas_usadas

    def obtem_linhas_usadas(self):
        linhas_de_alvo = self.obtem_linhas_de_alvo_usadas()
        linhas_de_controle = self.obtem_linhas_de_controle_usadas()

        linhas_usadas = linhas_de_alvo.union(linhas_de_controle)

        return linhas_usadas

    def inverte_controle_nas_linhas(self, linhas):
        for linha in linhas:
            for ctrl in self.controles:
                if linha == ctrl.linha:
                    ctrl.inverte_sinal()

    def roda_porta_permutacao(self, elementos):
        for i in range(len(elementos)):
            elem = elementos[i]
            # print('ELEM_ANTES -->', elem)

            if self.__deve_aplicar(elem):
                elementos[i] = self.__aplica(elem)

    def roda_porta_tabela_verdade(self, tabela):
        # print('T', tabela)

        for linha_tabela in tabela:
            alvo = self.alvos

            # print('L1', linha_tabela)

            if self.__deve_aplicar(linha_tabela):
                # print('aplicou')
                linha_tabela[alvo.linha] = int(not linha_tabela[alvo.linha])
                # print('L2', linha_tabela)

    def __deve_aplicar(self, elementos):
        for ctrl in self.controles:
            linha = ctrl.linha
            linha_mapeada = self.mapeamento[linha]
            valor = int(elementos[linha_mapeada])

            if ctrl.eh_sinal_positivo() and valor == 0:
                return False

            if ctrl.eh_sinal_negativo() and valor == 1:
                return False

        return True

    def __aplica(self, e):
        el = list(e)
        # print('\t\tVetor Elementos', el)

        linha_alvo = self.alvos.linha
        # print('\t\tLinha Alvo', linha_alvo)

        linha_alvo_mapeada = self.mapeamento[linha_alvo]
        # print('\t\tMapeamento', self.mapeamento)
        # print('\t\tLinha Alvo Mapeada', linha_alvo_mapeada)

        if el[linha_alvo_mapeada] == '0':
            el[linha_alvo_mapeada] = '1'
        elif el[linha_alvo_mapeada] == '1':
            el[linha_alvo_mapeada] = '0'
        else:
            raise Exception('Algum erro ocorreu!')

        resultado = ''.join(el)
        # print('ELEM_DEPOIS-->', resultado)

        return resultado

    def __repr__(self):
        s = f'T{self.tamanho_bits} '

        for ctrl in sorted(self.controles):
            s += f'{ctrl},'

        s += f'{self.alvos}'

        return s

    def __eq__(self, other):
        if not isinstance(other, self.__class__):
            return False

        if self.obtem_alvos() != other.obtem_alvos():
            return False

        if len(self.controles) != len(other.controles):
            return False

        if self.controles != other.controles:
            return False

        return True

    def __hash__(self):
        return id(self.alvos) * hash(self.controles) * self.tamanho_bits

    def __len__(self):
        return self.tamanho_bits
    

def gera_porta_circuito(expressao, alvo, prefixo='b', valor_inicial=0):
    partes = expressao.split('&')
    partes = [parte.strip().replace(' ', '') for parte in partes]

    alvos = list()

    if type(alvo) is int:
        alvos = Alvo(alvo)
    else:
        for a in alvo:
            um_alvo = Alvo(a)
            alvos.append(um_alvo)


    # constroi os controles
    controles = list()

    for parte in partes:
        ctrl_linha_str = parte.replace(prefixo, '')
        ctrl_sinal = True

        if parte.startswith('~'):
            ctrl_linha_str = parte[1 + len(prefixo):]
            ctrl_sinal = False

        if ctrl_linha_str != '':
            ctrl_linha = int(ctrl_linha_str)
            ctrl = Controle(ctrl_linha, ctrl_sinal)
            controles.append(ctrl)


    porta = Toffoli(alvos, controles)
    return porta


def gera_tabela_verdade(qtd_bits, ancillas=0):
    for linha in product([0, 1], repeat=qtd_bits):
        temp = list(linha)
        temp += [0] * ancillas
        yield temp


def gera_permutacao(qtd_bits):
    n = 2 ** qtd_bits
    estado = list()
    # estado = np.empty(n, dtype='object')

    for i in range(n):
        k = bin(i)[2:].zfill(qtd_bits)
        estado.append(k)
        # estado[i] = bin(i)[2:].zfill(qtd_bits)

    return estado


def inverte_elementos(elementos):
    for i in range(len(elementos)):
        elementos[i] = elementos[i][::-1]


class Circuito:
    def __init__(self, portas=None, qtd_vars=0, ancillas=0, mapa=None):
        """
            Recebe uma lista de portas e quantidade de variaveis.
        :param portas: Uma lista de Portas do tipo Fredkin ou Toffoli.
        :param qtd_vars: Uma quantidade inteira que representa a quantidade de variaveis.
        """
        self.bits_extras = ancillas
        self.tamanho_circuito = qtd_vars

        self.portas = portas
        if self.portas is None:
            self.portas = list()

        self.mapeamento = mapa
        if self.mapeamento is None:
            mapa = [(i, i) for i in range(self.tamanho_circuito)]
            self.mapeamento = dict(mapa)

        if self.tamanho_circuito is None:
            self.tamanho_circuito = self.__obtem_quantidade_de_variaveis()

        self.portas = self.__adiciona_portas(portas)

        ### apesar de achar que deve ser um atributo, demanda muito tempo gerar a tabela vdd a cada inicializacao
        # self.tabela_verdade = self.obtem_tabela_verdade()
        self.gc = self.custo_porta()
        self.qc = self.custo_quantico()

    def __adiciona_portas(self, info):
        if info is None:
            info = list()

        if isinstance(info, Circuito):
            info = info.portas

        if isinstance(info, tuple):
            info = list(info)

        if isinstance(info, list):
            portas = list()

            for p in info:
                ptemp = copy.deepcopy(p)
                ptemp.mapeamento = self.mapeamento
                portas.append(ptemp)

            return portas

        raise Exception('Circuito:: Operação não suportada!')

    def __obtem_quantidade_de_variaveis(self):
        linhas_usadas = self.obtem_linhas_usadas()
        return len(linhas_usadas)

    def obtem_mapeamentos(self):
        mapa = f'Mapeamento do Circuito: {self.mapeamento}'

        mapa += f'\nMapeamento das portas:'
        for porta in self.portas:
            mapa += f'\n\t{porta} ==> {porta.mapeamento}'

        return mapa

    def obtem_quantidade_de_bits(self):
        return self.tamanho_circuito + self.bits_extras

    def atualiza_informacoes(self):
        """
        Atualiza informações sobre o circuito.

        @return:
        """
        # self.tamanho_circuito = self.__obtem_quantidade_de_variaveis()
        self.tamanho_circuito = 1 + max(self.obtem_linhas_usadas(), default=0)
        self.gc = self.custo_porta()
        self.qc = self.custo_quantico()

    def obtem_copia(self):
        return copy.deepcopy(self)

    def reduz_linhas(self):
        mapeamento = dict()
        ini_vars = self.tamanho_circuito
        ini_ancs = self.bits_extras

        linhas = self.obtem_linhas_usadas()
        linhas = list(linhas)
        # logger.debug(f'Linhas: {linhas}')

        for i in range(0, ini_vars):
            if i not in linhas:
                self.tamanho_circuito -= 1
                continue

            mapeamento[i] = linhas.index(i)

        for i in range(ini_vars, ini_vars + ini_ancs):
            if i not in linhas:
                self.bits_extras -= 1
                continue

            mapeamento[i] = linhas.index(i)

        # logger.debug(f'MAP: {mapeamento}')

        for porta in self.portas:
            porta.modifica_linhas(mapeamento)

    def obtem_linhas_de_alvo_usadas(self):
        linhas_usadas = set()

        for porta in self.portas:
            linhas = porta.obtem_linhas_de_alvo_usadas()
            linhas_usadas = linhas_usadas.union(linhas)

        return linhas_usadas

    def obtem_linhas_de_controle_usadas(self):
        linhas_usadas = set()

        for porta in self.portas:
            linhas = porta.obtem_linhas_de_controle_usadas()
            linhas_usadas = linhas_usadas.union(linhas)

        return linhas_usadas

    def obtem_linhas_usadas(self):
        linhas_de_alvo = self.obtem_linhas_de_alvo_usadas()
        linhas_de_controle = self.obtem_linhas_de_controle_usadas()

        linhas_usadas = linhas_de_alvo.union(linhas_de_controle)
        return linhas_usadas

    def apaga_porta_por_indice(self, indice):
        del self.portas[indice]
        self.atualiza_informacoes()

    # def remove_porta(self, porta):
    #     self.portas.remove(porta)
    #     self.__atualiza_informacoes()

    def insere_porta_por_indice(self, indice, porta):
        self.portas.insert(indice, porta)
        self.atualiza_informacoes()

    # def adiciona_porta(self, porta):
    #    uma_porta = copy.deepcopy(porta)
    #    self.portas.append(uma_porta)
    #    self.__atualiza_informacoes()

    def custo_porta(self):
        return len(self.portas)

    def custo_quantico(self):
        custo_total = 0

        for porta in self.portas:
            custo_total += porta.calcula_custo_quantico()

        return custo_total

    def custo_cnot(self):
        custo_total = 0

        for porta in self.portas:
            # logger.debug(f'Porta: {porta}')
            custo = porta.calcula_custo_cnot()
            # logger.debug(f'Custo: {custo}')
            custo_total += custo
            # logger.debug(f'Total: {custo_total}')

        return custo_total

    def calcula_tamanho(self):
        linhas_usadas = self.obtem_linhas_usadas()
        return max(linhas_usadas)

    def obtem_permutacao(self, qtd_bits=None):
        if qtd_bits is None:
            qtd_bits = self.obtem_quantidade_de_bits()

        elementos = gera_permutacao(qtd_bits)
        # logger.debug(f'Permutacao Inicial: {elementos}')
        # logger.debug(f'Mapeamento: {self.mapeamento}')

        inverte_elementos(elementos)
        for porta in self.portas:
            # logger.debug(f'Aplicando {porta}')
            porta.roda_porta_permutacao(elementos)
            # logger.debug(f'Aplicado {elementos}')
        inverte_elementos(elementos)

        return tuple(elementos)

    def obtem_tabela_verdade(self, n=None, ancillas=None):
        if n is None:
            n = self.tamanho_circuito

        if ancillas is None:
            ancillas = self.bits_extras

        tab_vdd = gera_tabela_verdade(n, ancillas=ancillas)
        # tabela_verdade = list(tab_vdd)
        tabela_verdade = np.fromiter(tab_vdd, dtype=np.dtype((np.uint8, n)), count=2**n)
        # tabela_verdade = np.zeros(shape=(2**n, n), dtype=np.uint8)
        # for i, k in enumerate(tab_vdd):
        #     tabela_verdade[i] = k

        for porta in self.portas:
            porta.roda_porta_tabela_verdade(tabela_verdade)

        return tabela_verdade

    def obtem_portas_string(self):
        s = [str(porta) for porta in self.portas]
        return s

    def salva_em_arquivo(self, nome_do_arquivo=None):
        conteudo = str(self)

        if nome_do_arquivo is None:
            timestamp = time.time()
            nome_do_arquivo = Path(f'./data/circuitos/{timestamp}_GC-{self.gc}_QC-{self.qc}.tfc')

        # safety-check
        arquivo = Path(nome_do_arquivo)
        arquivo.parent.mkdir(parents=True, exist_ok=True)

        arquivo.write_text(conteudo)

    def __add__(self, other):
        if not isinstance(other, self.__class__):
            raise Exception('Tipo Circuito somente pode ser concatenado com outro de mesmo tipo.')

        # portas_invertidas = other.portas[::-1]
        # novas_portas = self.portas + portas_invertidas
        novas_portas = self.portas + other.portas
        nova_qtd_vars = max(self.tamanho_circuito, other.tamanho_circuito)
        novos_ancillas = max(self.bits_extras, other.bits_extras)

        novo_circuito = Circuito(novas_portas, nova_qtd_vars, novos_ancillas)

        # linhas_usadas = novo_circuito.__obtem_quantidade_de_variaveis()
        # novo_circuito.bits_extras = linhas_usadas - nova_qtd_vars

        # linhas_usadas = novo_circuito.obtem_linhas_usadas()
        # if len(linhas_usadas) > 0:
        #    novo_circuito.bits_extras = 1 + max(linhas_usadas) - nova_qtd_vars

        return novo_circuito

    def __repr__(self):
        s = ''
        s += f'# GATE_COUNT = {self.gc} \t\t QUANTUM_COST = {self.qc} \t\t ANCILLAS = {self.bits_extras}\n'

        # k = ','.join([f'b{i}' for i in sorted(self.obtem_linhas_usadas())])
        k = ','.join([f'b{i}' for i in range(self.tamanho_circuito + self.bits_extras)])
        s += f'.v {k}\n'
        s += f'.i {k}\n'
        s += f'.o {k}\n'

        s += 'BEGIN\n'
        for porta in self.portas:
            s += str(porta)
            s += '\n'

        s += 'END'
        return s

    def __eq__(self, other):
        if not isinstance(other, self.__class__):
            return False

        if self.tamanho_circuito != other.tamanho_circuito:
            return False

        if len(self.portas) != len(other.portas):
            return False

        if self.portas != other.portas:
            return False

        return True

    def __lt__(self, other):
        return self.tamanho_circuito < other.tamanho_circuito

    def __len__(self):
        return len(self.portas)

    def __iter__(self):
        for porta in self.portas:
            yield porta


def extrai_coluna_da_matriz(matriz, posicao):
    return [int(linha[posicao]) for linha in matriz]


def gera_circuito(expressao, alvo, ancillas, qtd_vars, prefixo='b'):
    if expressao.strip() == '':
        return Circuito(qtd_vars=qtd_vars, ancillas=ancillas)

    portas = extrai_portas(expressao, alvo)

    portas_circuito = list()
    for p in portas:
        porta_circuito = gera_porta_circuito(p, alvo, prefixo=prefixo)
        portas_circuito.append(porta_circuito)

    return Circuito(portas_circuito, qtd_vars, ancillas=ancillas)


def extrai_portas(expressao, alvo):
    # reduz espaços da expressão
    expressao = ' '.join(expressao.split())

    portas = list()

    if expressao == '1':
        # return [f'~b{alvo}']
        porta = ''
        portas.append(porta)
        return portas

    portas = expressao.split('^')
    portas = [porta.strip() for porta in portas]
    reduz_representacao_portas(portas)

    return portas


def reduz_representacao_portas(portas):
    identificador_de_not = '1'

    if identificador_de_not in portas:
        # qual a quantidade de negacoes na expressao?
        qtd = portas.count(identificador_de_not)

        # se for par, é só remover
        for _ in range(qtd):
            portas.remove(identificador_de_not)

        # caso seja impar, adiciona uma negacao na frente
        if qtd % 2 == 1:
            portas[0] = '~' + portas[0]

    # return portas


def gera_estados(n: int, alvos: List[int], prefixo: str = 'x'):
    """
    Gera a lista de estados referente ao circuito desejado.

    :param n: Indica a quantidade de linhas que o circuito possui.
    :param alvos: Indica as linhas que representam os alvos existentes.
    :param prefixo: Qual deve ser o prefixo dos estados gerados?
    :return: Uma lista de estados contendo todas as linhas que não são alvos.
    """
    estados = []

    for i in range(0, n + 0):
        estado = f'{prefixo}{i}'

        if i not in alvos:
            estados.append(estado)

    return estados


def processa_permutacao(g, n, estados, nivel=0, prefixo='x', alvo=0):
    return processa_permutacao_aux(g, n, estados, nivel, prefixo, alvo)

XOR = '⊕'
VAZIO = 'Ø'
AND = '.'

def substitui_simbolos(expressao):
    expressao = expressao.replace(AND, '&')
    expressao = expressao.replace(XOR, '^')
    expressao = expressao.replace(VAZIO, '')

    # inclui '~' das variaveis
    # expressao = 'b1 ^ 1' ==> '~b1'
    regex = r'[a-zA-Z]+[0-9]+\s+\^\s+1'
    variaveis = re.findall(regex, expressao)
    for s in variaveis:
        tt = s.replace(' ^ 1', '')
        tt = '~' + tt
        expressao = expressao.replace(s, tt)

    # sanitiza expressão
    expressao = ' ' + expressao.strip(' ') + ' '
    while '  ' in expressao:
        expressao = expressao.replace('  ', ' ')

    return expressao


def gera_string_binaria_equivalente(s, estados, n, gerar_negativos=False, alvo=0):
    variaveis = list()

    # gera uma representacao binaria do indice do espectro de reed muller
    r = f'{s:b}'.zfill(n)

    for j, x in enumerate(r):
        if x == '1':
            k = f'{estados[j]}'
            variaveis.append(k)
        elif gerar_negativos and x == '0':
            k = f'~{estados[j]}'
            variaveis.append(k)

    return variaveis


def transforma_saida_funcao(f):
    """
    In the S coding, the {0, 1} values corresponding to true and false minterms are
    respectively replaced by the {1, −1} values.
    """

    g = list()
    t = {0: 1, 1: -1}

    for i in f:
        e = t.get(i)

        if e:
            g.append(e)
        else:
            raise Exception("The function contains errors. The output must contain only 0's and 1's")

    return np.array(g, dtype=np.int8)


def gera_matriz_walsh_hadamard(n):
    """
    Walsh functions can be generated in a recursive way by using the Hadamard
    matrix (Hurst et al., 1985).

    Tw(0) = [1]
    Tw(n) = [ [Tw(n - 1), Tw(n - 1)], [Tw(n - 1), -Tw(n - 1)] ]
    """

    h0 = np.array([[1, 1], [1, -1]], dtype=np.int8)
    h = h0
    while len(h) < n:
        h = np.kron(h, h0)

    return h


def multiplica_matrizes(h, f):
    """
    add @measure to test performance on this function.

    Function Name       : multiplica_matrizes (with matmul)
    Elapsed time        :     0.0189 msecs
    Current memory usage:     0.0006 MB
    Peak                :     0.0009 MB

    Function Name       : multiplica_matrizes (with dot)
    Elapsed time        :     0.0285 msecs
    Current memory usage:     0.0015 MB
    Peak                :     0.0068 MB
    """

    s = np.matmul(h, f)
    return s


def aplica_funcao_s(coef):
    """
    000 --> 0 -->
    001 --> 1 --> x3
    010 --> 2 --> x2
    100 --> 3 --> x1
    ...
    111 --> 7 --> x1x2x3
    """

    coef = list(coef)
    n = int(log2(len(coef)))

    valor, pos = identifica_maior_magnitude(coef)
    r = f'{pos:b}'.zfill(n)

    negado = False
    if valor < 0:
        negado = True

    # [f'x{j+1}' if x == '1' else '' for j, x in enumerate('111')]

    linhas = list()
    for i in range(len(coef)):
        str_linha = f'{i:b}'.zfill(n)
        linha = list(str_linha)
        # logger.debug(f'antes == {linha=}', end='\t')

        for j, x in enumerate(r):
            if x == '1':
                if negado:
                    linha[j] = '0' if linha[j] == '1' else '1'

        # logger.debug(f'depois == {linha=}')
        linhas.append(linha)

    saida = list()
    for linha in linhas:
        t = list()

        for j, x in enumerate(r):
            if x == '1':
                k = int(linha[j])
                t.append(k)

        # logger.debug(f'T antes  > {t}')
        t = reduz_linha_xor(t)
        # logger.debug(f'T depois > {t}')
        saida.append(t)

    return saida


def reduz_linha_xor(lin):
    """
    x1 ^ x2 ==> x1 XOR x2
    """
    if len(lin) < 1:
        return 0
    ll = [str(x) for x in lin]
    tt = '^'.join(ll)
    res = eval(tt)
    return res


def executa_linha(f0, estados, nivel, alvo=0):
    f1 = transforma_saida_funcao(f0)
    # espacos = '   ' * nivel

    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.hadamard.html
    n = len(f0)
    wh = gera_matriz_walsh_hadamard(n)
    # logger.debug(f'WH = {wh}')

    s = multiplica_matrizes(wh, f1)

    fn = aplica_funcao_s(s)

    d = concordancia_de_funcoes(f0, fn)

    r0 = d.count(0)
    r1 = d.count(1)
    r = min(r0, r1) / len(fn)

    eq_d = gera_equacao_d(s, estados, alvo=alvo)
    # logger.debug(f'EQ = {eq_d}')

    return r, eq_d, d


def identifica_maior_magnitude(coeficiente_espectral):
    """
    We find coefficients with the largest magnitudes. In our example this condition is satisfied
    for the coefficient s1 = −6. Hence, f(x) = f(x3).

    Example: [2, −6, 2, 2, −2, −2, −2, −2]
    """

    magnitudes = coeficiente_espectral[:]
    magnitudes.sort(key=abs, reverse=True)

    maior_magnitude = magnitudes[0]
    posicao = coeficiente_espectral.index(maior_magnitude)

    return maior_magnitude, posicao



def gera_equacao_d(coef, estados, simbolo_xor='⊕', alvo=0):
    n = int(log2(len(coef)))
    coef = list(coef)
    variaveis = list()

    valor, pos = identifica_maior_magnitude(coef)
    eq = ''

    variaveis += gera_string_binaria_equivalente(pos, estados, n, gerar_negativos=False, alvo=alvo)

    if len(variaveis) > 0 and valor < 0:
        variaveis.append('1')

    if '1' in variaveis:
        variaveis2 = list()

        for v in variaveis:
            if v != '1':
                v = f'~{v}'
                variaveis2.append(v)

        variaveis = variaveis2

    eq += f' {simbolo_xor} '.join(variaveis)
    # eq += f' {simbolo_xor} '


    return eq.strip(f' {simbolo_xor} ')


def concordancia_de_funcoes(f1, f2):
    """
    Compute the function d(x) = f (x) ⊕ fs (x). The function d(x) indicates the number of
    agreements (disagreements) between f (x) and fs (x) at point x:

    d(x) = 1 for f(x) != fs(x).
    d(x) = 0 for f(x) == fs(x).
    """

    d = list()

    if len(f1) != len(f2):
        raise Exception("The function contains errors. Functions should have the same length.")

    for i in range(len(f1)):
        x1 = f1[i]
        x2 = f2[i]

        r = 1 if x1 != x2 else 0

        d.append(r)

    return d


def determina_quantidades_de_bits(d):
    return int(log2(len(d)))


def particiona_permutacao(dh, k=0, n=None):
    """
        Divide o DH particionando através do elemento fixo K.
    """
    if n is None:
        n = determina_quantidades_de_bits(dh)

    dh1 = list()
    dh2 = list()

    for i, h in enumerate(dh):
        r = f'{i:b}'.zfill(n)

        if r[k] == '0':
            dh1.append(h)
        else:
            dh2.append(h)

    return [dh1, dh2]


# n: quantidade de bits (inclui o alvo?) supomos que sim
def processa_permutacao_aux(g, n, estados, nivel=0, prefixo='x', alvo=0):
    # logger.debug(f'Entrando na recursão nível {nivel}')

    # se DH possui apenas 1, então termina (equação é igual a 1)
    if g.count(1) == len(g):
        # logger.debug(f'Encontrei uma G sem entrada nula')
        return f'1'

    # se DH está zerado, então termina (equação vazia)
    if [0] * len(g) == g:
        # logger.debug(f'Equação (zerada) == {g}')
        return f'{VAZIO}'

    if g.count(1) < 2:
        # logger.debug(f'Encontrei uma G com apenas uma entrada não nula')
        posicao = g.index(1)
        str_bin = gera_string_binaria_equivalente(posicao, estados, n, gerar_negativos=True, alvo=alvo)
        equacao = ' . '.join(str_bin)

        return equacao

    # se DH tem menos de dois elementos, então termina
    if len(g) < 2:
        # logger.debug(f'Equação (tam<2) == {g}')
        # return f'?'
        return f'1'

    metade_tamanho = len(g) // 2
    primeira_metade_g = g[0:metade_tamanho]
    segundo_metade_g = g[metade_tamanho:len(g)]
    if primeira_metade_g == segundo_metade_g:
        novos_estados = estados[:]
        del novos_estados[0]

        v = processa_permutacao_aux(g=primeira_metade_g, n=n - 1, estados=novos_estados, nivel=nivel + 1,
                                    prefixo=prefixo, alvo=alvo)

        return v

    espacos = '   ' * nivel
    _, eqd, dh = executa_linha(g, estados, nivel, alvo=alvo - nivel)

    # Equação total (ou final) --- resposta da recursão
    eqt = []

    # se a equação encontra neste nível da recursão é vazia, ou
    # se a equação é simplesmente igual a 1, dai retorna vazio
    # c.c. junta a equação encontrada neste nível com a equação anterior (encontrada nos niveis acima)
    if eqd == '' or eqd.strip() == '1':
        eqd = f'{VAZIO}'
    else:
        eqt.append(eqd)

    if dh.count(1) == 1:
        # logger.debug(f'\t\t\tDH:{dh}')

        # logger.debug(f'\t*\tfinal: {v0} &\t {v1}')
        idx = dh.index(1)
        valor_str = f'{idx:b}'.zfill(n)
        # logger.debug(f'IDX - valor str:{valor_str}')

        posicao = dh.index(1)
        str_bin = gera_string_binaria_equivalente(posicao, estados, n, gerar_negativos=True, alvo=alvo)
        eqd = ' . '.join(str_bin)

        eqt.append(eqd)

        eq = f' {XOR} '.join(eqt)
        return eq

    n -= 1

    ##### REMOVER DEPOIS
    eq1 = ''
    eq2 = ''
    #####

    # logger.debug(f'n:{n}')
    if dh != [0] * n:
        # se novo DH não está zerado, particiona
        g1, g2 = particiona_permutacao(dh, k=0)

        # logger.debug(f'{espacos}EQ_D{nivel} = {eqd}')
        # logger.debug(f'{espacos}FP{nivel} = {eqd}')
        # logger.debug(f'{espacos}Particionando F{nivel} = {dh} em F{nivel + 1} = {g1} e F{nivel + 1}\' = {g2}')

        # estados_temp = estados[:alvo] + estados[alvo+1:]
        estados_temp = estados[1:]
        # logger.debug(f'{L=} {i=} {Ltemp=}')

        ind = estados[0]

        # logger.debug(f'       tbl_vdd = {dh} \t\t qtd_1s = {dh.count(1)}')
        # logger.debug(f'       tbl_g1 = {g1} \t\t qtd_1s = {g1.count(1)}')
        # logger.debug(f'       tbl_g2 = {g2} \t\t qtd_1s = {g2.count(1)}')

        # antes o parâmetro era n-1
        v0 = processa_permutacao_aux(g1, n, estados_temp, nivel + 1, prefixo, alvo - nivel)
        # logger.debug(f'{"---" * nivel}expr_F{nivel + 1} = {v0}')

        # antes o parâmetro era n-1
        v1 = processa_permutacao_aux(g2, n, estados_temp, nivel + 1, prefixo, alvo - nivel)
        # logger.debug(f'{"---" * nivel}expr_F{nivel + 1}\'= {v1}')

        # logger.debug(f'n:{n}')

        # logger.debug(f'V0 = {v0}')
        # logger.debug(f'V1 = {v1}')

        if v0 == '1':
            eq1 = f' ~{ind} '
            eqt.append(eq1)

        elif v0 != f'{VAZIO}':
            eq1t = list()
            for variavel in v0.split(f'{XOR}'):
                variavel = variavel.strip()
                eq1t.append(f' ~{ind} . {variavel}')

            eq1 = f' {XOR} '.join(eq1t)
            # logger.debug(f'EQ1 ==> {eq1} :: EQ1T ==> {eq1t}')

            # eq1 = f' ~{ind} . ( ' + str(v0) + ' )'
            eqt.append(eq1)

        if v1 == '1':
            eq2 = f' {ind} '
            eqt.append(eq2)

        elif v1 != f'{VAZIO}':
            eq2t = list()
            for variavel in v1.split(f'{XOR}'):
                variavel = variavel.strip()
                eq2t.append(f' {ind} . {variavel}')

            eq2 = f' {XOR} '.join(eq2t)
            # logger.debug(f'EQ2 ==> {eq2} :: EQ2T ==> {eq2t}')

            # eq2 = f' {ind} . ( ' + str(v1) + ' )'
            eqt.append(eq2)

    # logger.debug(f'{eq1=}')
    # logger.debug(f'{eq2=}')
    # logger.debug(f'{eqt=}')

    eq = f' {XOR} '.join(eqt)
    # logger.debug(f'{espacos}FT{nivel} = {eq}')

    return eq


def executa_sintese(n, tabela_saida, prefixo='b'):
    # verifica se a quantidad de linhas da entrada difere da quantidade de linhas de saída
    # if len(tabela_entrada) != len(tabela_saida):
    if (2 ** n) != len(tabela_saida):
        raise Exception('Tabela de saída incogruente com a tabela de entrada.')

    # infere alguns dados a partir das tabelas de entrada e saída
    # n = len(tabela_entrada[0])

    # 2024-07-19: gambiarra?
    if isinstance(tabela_saida[0], int):
        tabela_saida = [[i] for i in tabela_saida]
        # tabela_saida = [tabela_saida]

    ancillas = len(tabela_saida[0])

    # extrai todas as funções possiveis, dada uma tabela de saída (considerando a entrada 000, 001, 010, ... 110, 111)
    ## F_i = a i-ésima saída da função original (diretamente da tabela de saída informada)
    ## FXOR_i = a i-ésima função F_i aplicando um XOR com o valor do alvo, ou seja, FXOR = F_i XOR i
    funcoes = list()
    for i in range(ancillas):
        # extrai primeiramente os valores da i-ésima coluna
        f = extrai_coluna_da_matriz(tabela_saida, i)
        # f_alvo = extrai_coluna_da_matriz(tabela_entrada, i)
        # fxor = gera_f1_xor(f, f_alvo)

        # salva a F encontrada
        funcoes.append(f)

    # gera o circuito de cada expressao reed-muller, para cada funcao encontrada no passo anterior
    circuitos = list()
    for i, fxor in enumerate(funcoes):
        # gera a lista de estados para uma determinada linha de alvo
        linha_de_alvo = [n + i]
        estados = gera_estados(n, linha_de_alvo, prefixo=prefixo)
        # gera a expressao
        expressao = processa_permutacao(fxor, n, estados, prefixo=prefixo, alvo=i)
        # limpa a expressao gerada: remove espacos desncessarios, parenteses, etc
        expressao_tratada = substitui_simbolos(expressao)
        expressao_tratada = expressao_tratada.replace('(', '').replace(')', '').strip()
        # gera um circuito equivalente a partir da expressao encontrada
        circuito = gera_circuito(expressao_tratada, alvo=linha_de_alvo, ancillas=ancillas, qtd_vars=n,
                                 prefixo=prefixo)
        circuitos.append(circuito)

    circuito_final = Circuito(qtd_vars=n, ancillas=ancillas)
    for c in circuitos:
        circuito_final = circuito_final + c

    # logger.debug(f'Circuito Final:\n{circuito_final}')
    # circuito_final.reduz_linhas()
    return circuito_final


def main():
    # Entrada Padrão (3 bits)
    # entrada = [
    #     [0, 0, 0],
    #     [0, 0, 1],
    #     [0, 1, 0],
    #     [0, 1, 1],
    #     [1, 0, 0],
    #     [1, 0, 1],
    #     [1, 1, 0],
    #     [1, 1, 1],
    # ]

    # Saida aleatória 01
    # saida = [
    #     [0, 1],
    #     [0, 0],
    #     [1, 1],
    #     [1, 0],
    #     [1, 1],
    #     [0, 0],
    #     [0, 1],
    #     [1, 0],
    # ]

    # https://iopscience.iop.org/article/10.1088/2058-9565/acaf9d/meta
    # saida = [
    #     [0, 1, 0],
    #     [1, 0, 0],
    #     [1, 0, 0],
    #     [0, 1, 0],
    #     [1, 1, 0],
    #     [1, 0, 0],
    #     [0, 0, 0],
    #     [0, 1, 1],
    # ]

    # https://www.researchgate.net/profile/Stefano-Mancuso-2/publication/320674387_Computers_from_Plants_We_Never_Made_Speculations/links/5abf6d8aa6fdcccda65ac970/Computers-from-Plants-We-Never-Made-Speculations.pdf#page=120
    # saida = [
    #     [1],
    #     [1],
    #     [1],
    #     [1],
    #     [0],
    #     [0],
    #     [0],
    #     [1],
    # ]

    saida = [
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
    ]
    circ = executa_sintese(n=3, tabela_saida=saida)
    print(circ.__repr__())
    return

if __name__ == "__main__":
    main()

# GATE_COUNT = 2 		 QUANTUM_COST = 6 		 ANCILLAS = 1
.v b0,b1,b2,b3
.i b0,b1,b2,b3
.o b0,b1,b2,b3
BEGIN
T2 b2,b3
T3 b0,b1,b3
END


In [8]:
from qiskit import *
from math import log2, ceil
from qiskit.circuit.library import XGate, UnitaryGate, MCXGate
from qiskit.circuit import Gate
import re


def encode_table(l: list[int], size: int) -> list[str]:
    """Function encodes integers in a given list into their binary representation with `size`-bits.

    Exemple:
    -
    `[4,2,3,1,6,5,8,7] -> ['0100', '0010', '0011', '0001', '0110', '0101', '1000', '0111']`

    Args:
        l (list[int]): list of integers to be encoded.
        size (int): how many bits elements should have in it's encoding.

    Returns:
        encoded_l(list[str]): a list of strings containing the binary encoding with size-bits of the numbers in l. 
    """
    encoded_l = []
    for num in l:
        encoded_l.append((bin(num)[2:]).rjust(size,"0"))
    return encoded_l


def generate_control_strings(size: int) -> list[str]:
    """Function generates a list of binary strings from `0` to `size`.

    Exemple:
    -
    `size = 6 -> ['000', '001', '010', '011', '100', '101']`

    Args:
        size (int): number elements to be generated (list size).

    Returns:
        c_strings(list[str]): a list of binary strings from `0-size`.
    """
    c_strings = []
    string_size = ceil(log2(size))
    for i in range(size):
        c_strings.append((bin(i)[2:]).rjust(string_size,"0"))
    return c_strings


def x_data_gates(l: list[str], size: int) -> list[Gate]:
    """Generates quantum gates from a list of binary strings,
    the ih-gate action is to set initialized qubits to the i-th string in the given list.

    Exemple:
    -  
    `xor_data_gates(['0100'], 4)`

    `|0> -> |4>`
     ____
    |0   |     q0 -----
    |1 L |  =  q1 -----
    |2 0 |     q2 --X--
    |3___|     q3 -----

    Args:
        l (list[str]): a list of binary strings.
        size (int): the number of bits the gate should affect.
    
    Returns:
        circuits(list[Gate]): a list of quantum gates, the ih-gate sets `|0>` to the bit-string from the ih-element in `l`.
    """
    circuits = []
    for bit_string in l:
        qc = QuantumCircuit(size)
        for bit in range(len(bit_string)):
            if bit_string[bit] == "1":
                qc.x(abs(bit-size)-1)
        circuits.append(qc.to_gate())

    for i in range(len(circuits)):
        circuits[i].name = f"L{i}"
    return circuits


def calculate_exp_table(W: int, a: int, N: int, only_odds: bool = False) -> list[int]:
    """Given a base `a`, a modulo `N` and a window size `W` calculates the table
    `tbl[d] = a^d mod N, d = 1,2,3,...,2^W-1`

    Exemple:
    -
    `calculate_exp_table(2,3,31)` -> `[1, 3, 9, 27]`
    `calculate_exp_table(2,3,31,True)` -> `[0, 3, 0, 27]`

    Args:
        W (int): window size.
        a (int): base of the exponentiation.
        N (int): modulo of the exponentiation.
        only_odds (bool): define if calculates the operation for all values of `d` or only when `d` is odd.
    
    Returns:
        exp_table(list[int]): a list with the results from the exponentiation.
    """
    exp_table = [0]*((1<<W))
    (step, start) = (2,1) if only_odds else (1,0)
    for d in range(start,(1<<W),step):
        exp_table[d] = ((a**d) % N)
    return exp_table

def calculate_mult_table(W: int, a: int, N: int) -> list[int]:
    """Given a base `a`, a modulo `N` and a window size `W` calculates the table
    `tbl[d] = a*d mod N, d = 1,2,3,...,2^W-1`

    Exemple:
    -
    `calculate_exp_table(2,3,31)` -> `[0, 2, 4, 6, 8, 10, 12, 14]`

    Args:
        W (int): window size.
        a (int): base of the multiplication.
        N (int): modulo of the operation.
    
    Returns:
        mult_table(list[int]): a list with the results from the modular multiplication.
    """
    mult_table = []
    for d in range(1<<W):
        mult_table.append((a*d) % N)
    return mult_table


def get_output_string(bin_list: list[str], size: int) -> list[str]:
    """
    separates the outputs by bit
    ex:
    `[0100, 1000, 0010, 0000, 0101]` -> `[[01000], [10001], [00100], [00001]]`
    -> `[[[0],[1],[0],[0],[0]], [[[1],[0],[0],[0],[1]], [[[0],[0],[1],[0],[0]], [[[0],[0],[0],[0],[1]]]`
    """
    new_list = ['']*size
    for i in range(size):
        for j in range(len(bin_list)):
            new_list[i] += bin_list[j][i]
    new_list = new_list[::-1]
    LL=[]
    for x in new_list:
        L=[]
        for y in x:
            L.append([int(y)])
        LL.append(L)
    return LL


def tfc_str_to_qiskit(tfc_str, num_ctrl_qubits):
    """
    Converts a quantum circuit described in TFC format string to a Qiskit QuantumCircuit.

    Args:
        tfc_str (str): String containing the TFC circuit description
        num_ctrl_qubits (int): Number of control qubits in the circuit

    Returns:
        QuantumCircuit: The equivalent Qiskit circuit
    """
    qubit_map = {}
    next_qubit_index = 0
    num_classical_bits = 0
    circuit_lines = []

    lines = [line.strip() for line in tfc_str.split('\n') if line.strip()]

    in_begin_section = False
    for line in lines:
        if not line or line.startswith('#'):
            continue

        if line.startswith('.v'):
            virtual_qubits = [q.strip() for q in line.split(' ')[1].split(',')]
            for q_name in virtual_qubits:
                if q_name.startswith('b') or q_name.startswith('s'):
                    qubit_map[q_name] = next_qubit_index
                    next_qubit_index += 1
        elif line.startswith('.o'):
            output_qubit_names = [q.strip() for q in line.split(' ')[1].split(',')]
            num_classical_bits = len(output_qubit_names)
        elif line == 'BEGIN':
            in_begin_section = True
        elif line == 'END':
            in_begin_section = False
        elif in_begin_section:
            circuit_lines.append(line)

    actual_num_qubits = max(qubit_map.values()) + 1 if qubit_map else 0

    qr = QuantumRegister(actual_num_qubits, 'q')
    qc = QuantumCircuit(qr)

    print(f"Total Quantum Qubits: {actual_num_qubits}")
    print(f"Total Classical Bits (for outputs): {num_classical_bits}")
    print("Qubit Mapping:", qubit_map)

    for gate_line in circuit_lines:
        match = re.match(r'(T\d+)\s(.*)', gate_line)
        if not match:
            print(f"Warning: Could not parse line: {gate_line}")
            continue

        gate_type = match.group(1)
        qubit_names_str = match.group(2)
        qubit_names = [q.strip() for q in qubit_names_str.split(',')]

        controls = []
        target_idx = None
        control_states_str = ""

        # Process target qubit
        target_name_raw = qubit_names[-1]
        if target_name_raw.endswith("'"):
            print(f"Warning: Target qubit '{target_name_raw}' is negated. Treating as non-negated.")
            target_name = target_name_raw[:-1]
        else:
            target_name = target_name_raw

        target_idx = qubit_map.get(target_name)
        if target_idx is None:
            print(f"Error: Target qubit '{target_name}' not found in qubit map for line: {gate_line}")
            continue

        # Process control qubits
        for q_name_raw in qubit_names[:-1]:
            is_negated = False
            q_name = q_name_raw
            if q_name_raw.endswith("'"):
                is_negated = True
                q_name = q_name_raw[:-1]

            control_idx = qubit_map.get(q_name)
            if control_idx is None:
                print(f"Error: Control qubit '{q_name}' not found in qubit map for line: {gate_line}")
                continue

            controls.append(qr[num_ctrl_qubits-control_idx-1])
            control_states_str += '0' if is_negated else '1'

        # Apply gates
        if gate_type == 'T1':
            qc.x(qr[target_idx])

        elif gate_type == 'T2':
            if len(controls) != 1:
                print(f"Warning: T2 gate expects 1 control, found {len(controls)} for {gate_line}. Skipping.")
                continue
            
            if control_states_str[0] == '0':
                qc.x(controls[0])
            qc.cx(controls[0], qr[target_idx])
            if control_states_str[0] == '0':
                qc.x(controls[0])

        elif gate_type == 'T3':
            if len(controls) != 2:
                print(f"Warning: T3 gate expects 2 controls, found {len(controls)} for {gate_line}. Skipping.")
                continue
            
            temp_controls = list(controls)
            for i, control_state in enumerate(control_states_str):
                if control_state == '0':
                    qc.x(temp_controls[i])

            qc.ccx(temp_controls[0], temp_controls[1], qr[target_idx])

            for i, control_state in enumerate(control_states_str):
                if control_state == '0':
                    qc.x(temp_controls[i])

        elif gate_type.startswith('T'):
            num_involved_qubits = int(gate_type[1:])
            if len(controls) + 1 != num_involved_qubits:
                print(f"Warning: Gate {gate_type} in line '{gate_line}' implies {num_involved_qubits} qubits, but found {len(controls)+1}.")
            
            mcx_gate = MCXGate(len(controls), ctrl_state=control_states_str[::-1])
            all_involved_qubits = controls + [qr[target_idx]]
            qc.append(mcx_gate, all_involved_qubits)

        else:
            print(f"Warning: Unknown gate type {gate_type} in line: {gate_line}")

    return qc


def transform_to_permutation(orig_list: list[int], n_bits: int) -> list[int]:
    """ get unique permutation of elements from original list
    retulting list is a permutation where the last n_bits from each element represent the original value
    and the first bits are used to ensure uniqueness of the permutation.
    The first bits can also be represented as the i-th appearance of that number.
    """
    new_list = []
    appearance = {key: 0 for key in orig_list}
    for i in orig_list:
        appearance[i] += 1
        if appearance[i] <= 1:
            new_list.append(i)
        else:
            new_number = i + (1 << n_bits) * (appearance[i] - 1)
            new_list.append(new_number)
    return new_list


def complete_permutation(orig_list: list[int], n_bits: int) -> list[int]:
    """Completes the permutation by adding missing elements to the original list.
    The resulting list will contain all integers from 0 to 2^n_bits -
    """
    for i in range(1<<n_bits):
        if i not in orig_list:
            orig_list.append(i)
    return orig_list


def build_index_circ(i: int, n: int, reg: QuantumRegister, aux: QuantumRegister, target: QuantumRegister, x_circs: list[QuantumCircuit], qc: QuantumCircuit):
    if i == n:
        return qc
    if i == 0:
        qc.ccx(reg[i], aux[i], aux[i+1], ctrl_state="10")
    else:
        qc.ccx(reg[i], aux[i], aux[i+1], "10")
    
    build_index_circ(i+1, n, reg, aux, target, x_circs, qc)
    
    if i == n-1:
        qc.append(x_circs[0].control(1), aux[i+1:i+2] + target[:])
        x_circs.pop(0)
        qc.barrier()

    qc.cx(aux[i], aux[i+1])

    if i == n-1:
        qc.append(x_circs[0].control(1), aux[i+1:i+2] + target[:])
        x_circs.pop(0)
        qc.barrier()

    build_index_circ(i+1, n, reg, aux, target, x_circs, qc)

    if i == 0:
        qc.ccx(reg[i], aux[i], aux[i+1])
    else:
        qc.ccx(reg[i], aux[i], aux[i+1])

    return qc


def compute_lookup_table(window_size: int, outBits: int, l: list[int], optimization: int = 0) -> QuantumCircuit:
    """Computes the lookup-table(QROM)`[1]`, the circuit takes an input `a` and has an effect of XOR'ing 
    the corresponding a-th value of the list `l` into the `outBits` output register.

    Exemple:
    -
    W = 6
    l = any integer list of size 8

    a0 ---○----○----○----○----●----●----●----●----
          |    |    |    |    |    |    |    |
    a1 ---○----○----●----●----○----○----●----●----
          |    |    |    |    |    |    |    |
    a2 ---○----●----○----●----○----●----○----●----
          |    |    |    |    |    |    |    |
    c  ---●----●----●----●----●----●----●----●----
          |    |    |    |    |    |    |    |
    W0 --| |--| |--| |--| |--| |--| |--| |--| |---
         | |  | |  | |  | |  | |  | |  | |  | |
    W1 --| |--| |--| |--| |--| |--| |--| |--| |---
         | |  | |  | |  | |  | |  | |  | |  | |
    W2 --|L|--|L|--|L|--|L|--|L|--|L|--|L|--|L|---
         |0|  |1|  |2|  |3|  |4|  |5|  |6|  |7|
    W3 --| |--| |--| |--| |--| |--| |--| |--| |---
         | |  | |  | |  | |  | |  | |  | |  | |
    W4 --| |--| |--| |--| |--| |--| |--| |--| |---
         | |  | |  | |  | |  | |  | |  | |  | |
    W5 --|_|--|_|--|_|--|_|--|_|--|_|--|_|--|_|---

    Complexity:
    -
    Number of bits per register:

    - `o`, output register, takes `outBits` bits.
    - `w`, input register (window), takes `⌈log2|l|⌉` bits.

    Args:
        outBits (int): output size in bits.
        l (list[int]): list to be computed.
        optimization (int): the level of optimization the circuit should have, `default = 0`.
    
    Returns:
        quantum_circuit(QuantumCircuit): the quantum circuit implementing the lookup table (QROM).

    Reference:
    -
    [1]Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity.
    Ryan Babbush, Craig Gidney, Dominic W. Berry, Nathan Wiebe, Jarrod McClean, Alexandru Paler, Austin Fowler, and Hartmut Neven
    """

    w = QuantumRegister(window_size, name="w")
    o = QuantumRegister(outBits, name="out")
    
    e_table = encode_table(l, outBits)
    
    match optimization:
        case 0:
            x_circs = x_data_gates(e_table, outBits)
            anc = QuantumRegister(window_size+1, "anc")
            quantum_circuit = QuantumCircuit(w, anc, o)
            quantum_circuit.name = "QROM"
            quantum_circuit.x(anc[0])

            build_index_circ(0, window_size, w, anc, o, x_circs, quantum_circuit)

        case 1:
            quantum_circuit = QuantumCircuit(w, o)
            quantum_circuit.name = "QROM"

            output_str = get_output_string(e_table,outBits)

            for i in range(len(output_str)):
                tfc_circ_str = executa_sintese(n=window_size, tabela_saida=output_str[i]).__repr__()
                quantum_circuit.append(tfc_str_to_qiskit(tfc_circ_str, window_size), w[:] + o[i:i+1])

    return quantum_circuit


def uncompute_lookup(in_size: int, table_size: int) -> QuantumCircuit:
    return

In [9]:
from qiskit import *
from math import log2

def win_add_mod(N: int, w: int, n: int, k: int) -> QuantumCircuit:
    """
    Args:
        N (int): the modulus.
        w (int): the addition window size.
        n (int): bit size of the operand.
        k (int): constant to multiply y.
        
    Returns:
        quantum_circuit (QuantumCircuit): the plus equal product mod `x += ky mod N` circuit.
    """
    #ideal w = lg n

    reg_y = QuantumRegister(n, 'y')
    reg_t = QuantumRegister(n, 'target')
    reg_o = QuantumRegister(n+1, 'out')
    reg_anc = QuantumRegister(n+2, 'anc')


    quantum_circuit = QuantumCircuit(reg_anc,reg_y, reg_t, reg_o, name="win_add_mod")


    #anc, a, b, cO, n, help

    #for each window i  "range(0, len(y), w)"
        #win = y[i:i + w]
        #compute the lookup table (j*k*2^i mod N for j in range(2^w))
        #target += table[win]
    for i in range(0, n, w):
        table = [j * k * 2**i % N for j in range(2**w)]
        qrom = compute_lookup_table(w, n, table, optimization=1)
        quantum_circuit.append(qrom, reg_y[i:i + w] + reg_t[:])
        quantum_circuit.append(mod_adder_CDKM_VBE(n, N), reg_anc[0:1] + reg_t[:] + reg_o[:] + reg_anc[1:])
        quantum_circuit.append(qrom.inverse(), reg_y[i:i + w] + reg_t[:])
        
        #xor qrom into targer
        #add target into out
        #undo qrom xor
        #repeat

    return quantum_circuit




In [10]:
from qiskit import *

"""
pseudo code:
def times_equal_windowed(target: Quint, k: int, window: int):
    assert k % 2 == 1  # Even factors aren't reversible.
    k %= 2**len(target)  # Normalize factor.
    if k == 1:
        return
    table = LookupTable([
        (j * k) >> window
        for j in range(2**window)
    ])
    for i in range(0, len(target), window)[::-1]:
        w = target[i:i + window]
        target[i + window:] += table[w]

        # Recursively fix up the window.
        times_equal_windowed(w, k, window=1)
"""

#like shown in figure 6, the modular multiplication can be reduced into modular product additions, so, to do x *= k mod N
#we can simply do use the input a and the constant k in the plus equals product, the output will be a*k mod N
#then we use that result as the input and target a with k = (-k)^-1, so that the register will be 0
#we then swap the two registers, having the following result: |a> -> |a*k mod N> we use another n bit register to store partial results.


def modinv(a: int, n: int) -> int:
    t, new_t = 0, 1
    r, new_r = n, a
    while new_r != 0:
        quotient = r // new_r
        t, new_t = new_t, t - quotient * new_t
        r, new_r = new_r, r - quotient * new_r
    if r > 1:
        return None
    if t < 0:
        t = t + n
    return t


def win_mult_mod(N: int, w: int, n: int, k: int ) -> QuantumCircuit:

    reg_a = QuantumRegister(n, 'a')
    reg_temp = QuantumRegister(n, 'temp')
    reg_help = QuantumRegister(1, 'help') # used as cOut for adder
    reg_anc = QuantumRegister(2*n+2, 'anc')

    quantum_circuit = QuantumCircuit(reg_a, reg_temp, reg_anc, reg_help, name="win_mult_mod")

    quantum_circuit.append(win_add_mod(N, w, n, k), reg_anc[0:n+2] + reg_a[:] + reg_anc[n+2:] + reg_temp[:] + reg_help[:])
    quantum_circuit.append(win_add_mod(N, w, n, modinv(k, N)).inverse(), reg_anc[0:n+2] + reg_temp[:] + reg_anc[n+2:] + reg_a[:] + reg_help[:])
    quantum_circuit.swap(reg_a, reg_temp)

    #win add where y = reg a, out = reg temp, k = k
    #win add where y = reg temp, out = reg a, k = (-k)^-1
    #swap reg a, reg temp
    return quantum_circuit

In [11]:
import sympy

def approx_mod_exp(N:int, f:int, g:int, exp_size:int):
    """
    N: mod
    g: exp base for g^e mod N
    exp_size: exponent size in bits
    f: bits to keep in during approximation ~loglogN
    """
    n = int(log2(N)) + 1
    #P, L, d, max_ell = get_primes(N, exp_size, f) #prime set, pset product, P mod deviation, max bits used from p in P

    ###--------------------------###
    P = [53,3823,4969]
    L = math.prod(P)
    max_ell = max(p.bit_length() for p in P)
    ###--------------------------###

    t = n - f #number of bits to be truncated
    #u, C = get_constants(L, P, N, f) #get lists containing constans uj and Cjk

    assert modular_deviation(L, N) < 2**(-f), "Modular deviation should be less than 2^(-f)"
    assert L >= N**exp_size, "2"
    assert (L % N) < (N >> f), "3"
    assert L == lcm(*P) and L >= N**exp_size and (L % N) < (N >> f), "L should be a prime prod, grater than N^m and its modulo N should be less than the f truncation of N"
    

    #quantum circuit definition
    #sizes may change later
    reg_e = QuantumRegister(exp_size, "reg_e") # exponent reg
    reg_out = QuantumRegister(f, "reg_out") # output reg
    reg_res = QuantumRegister(max_ell, "reg_res") # residue reg
    reg_aux = QuantumRegister(3*max_ell+3, "reg_aux") # auxiliar reg, may change later
    quantum_circuit = QuantumCircuit(reg_e, reg_out, reg_res, reg_aux)

    quantum_circuit.h(reg_e)

    for j, p in enumerate(P):
        # sets residue reg to 1
        quantum_circuit.reset(reg_res)
        quantum_circuit.x(reg_res[0])

        # calculates rj through iterating in r_jk accumulating in qres
        for k in range(exp_size):
            r_jk = pow(g, 1<<k, N) % p #calculates current residue
            #control mult mod qres = (qres * r_jk) % p
            #controlled by reg_exp[k] 
            mult = win_mult_mod(p, 1, max_ell, r_jk)
            quantum_circuit.append(mult, reg_res[:] + reg_aux[:])
        
        u = (L // p) * pow(L// p, -1, p)
        # calculates out by the given formula V = (sum(j to lenP)sum(k to ell)rjk*Cjk) mod (N>>t)
        for k in range(max_ell):
            #do control add mod q_out = (q_out + Cjk) % (N >> t)
            #controlled by reg_res[k]
            print(j, k)
            c = (((u<<k) % L % N) >> t) % (N >> t)
            adder = adder_mod(reg_out.size, c, N>>t, True)
            quantum_circuit.append(adder, reg_res[k:k+1] + reg_out[:] + reg_aux[0:2])

    # final bit shift, Q_total << t, can be deduced in post processing
    #bit_shift = naive_bit_shift(t, reg_out.size)
    #quantum_circuit.append(bit_shift, reg_out[:])

    return quantum_circuit

approx_mod_exp(1001, 6, 2, 3).draw("mpl")

Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical 

Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical Bits (for outputs): 2
Qubit Mapping: {'b0': 0, 'b1': 1}
Total Quantum Qubits: 2
Total Classical 

In [14]:
c = approx_mod_exp(1001, 6, 2, 3)
reg_e = QuantumRegister(3)
reg_out = QuantumRegister(6)
reg_res = QuantumRegister(13)
reg_aux = QuantumRegister(42)
clas = ClassicalRegister(6)
qc = QuantumCircuit(reg_e, reg_out, reg_res, reg_aux, clas)
qc.append(c, reg_e[:] + reg_out[:] + reg_res[:] + reg_aux[:])
qc.measure(reg_out, clas)
qc.draw("mpl")

In [15]:
from qiskit_aer import AerSimulator


backend1 = AerSimulator()
backend1.set_max_qubits(80)
qc1 = transpile(qc, backend=backend1)

from qiskit_ibm_runtime import SamplerV2 as Sampler

sampler = Sampler(backend1)
job = sampler.run([qc1], shots=100)

counts = job.result()[0].data.clas.get_int_counts()

Simulation failed and returned the following error message:
ERROR:  [Experiment 0] Insufficient memory to run circuit circuit-442784-442794 using the statevector simulator. Required memory: 35184372088832M, max memory: 16256M


ValueError: could not broadcast input array from shape (0,1) into shape (100,1)